In [1]:
import os
import glob
import re
import numpy as np
import pandas as pd
import datetime
from functools import reduce

## Verification of the goodness of the data

In [2]:
years = ['2016-2017', '2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022']


path = '/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/'

sexes = ['men', 'women']

loops = {'sprint':['loop_1', 'loop_2'], 'mass_start': ['loop_1', 'loop_2', 'loop_3', 'loop_4'],\
    'individual': ['loop_1', 'loop_2', 'loop_3', 'loop_4'], 'mass_start': ['loop_1', 'loop_2', 'loop_3', 'loop_4'],\
        'pursuit':['loop_1', 'loop_2', 'loop_3', 'loop_4'], 'sprint_2':['loop_1', 'loop_2', 'loop_3', 'loop_4'],\
            'pursuit_2': ['loop_1', 'loop_2', 'loop_3', 'loop_4']}




In [8]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
i = 0
for year in years:
    for sex in sexes:
        locations = os.listdir(path + f'{sex}/{year}_{sex[0]}')
        locations = np.array(locations)[np.array(locations) !=  '.DS_Store']
        for location in locations:
            format_path = glob.glob((path + f'{sex}/{year}_{sex[0]}/{location}/intermediates/*'))
            formats = [f.split('/')[-1] for f in format_path]
            for format in formats:            
                files = glob.glob(path + f'{sex}/{year}_{sex[0]}/{location}/intermediates/{format}/*.txt')
                for file in files:
                    try:
                        pd.read_csv(file, sep = '\t')
                    except:
                        i += 1
                        print(file, i)
                        

In [ ]:
i = 0
for year in years:
    for sex in sexes:
        locations = os.listdir(path + f'{sex}/{year}_{sex[0]}')
        locations = np.array(locations)[np.array(locations) !=  '.DS_Store']
        for location in locations:
            format_path = glob.glob((path + f'{sex}/{year}_{sex[0]}/{location}/intermediates/*'))
            formats = [f.split('/')[-1] for f in format_path]
            for format in formats:
                for loop in loops[format]:
                    files_loop = glob.glob(path + f'{sex}/{year}_{sex[0]}/{location}/{loop}_{location}_{format}.txt')                
                for file in files_loop:
                    
                    try:
                        pd.read_csv(file, sep = '\t')
                    except:
                        i += 1
                        print(file, i)
                        

In [5]:

i = 0
for year in years:
    for sex in sexes:
        locations = os.listdir(path + f'{sex}/{year}_{sex[0]}')
        locations = np.array(locations)[np.array(locations) !=  '.DS_Store']
        for location in locations:
            format_path = glob.glob((path + f'{sex}/{year}_{sex[0]}/{location}/intermediates/*'))
            formats = [f.split('/')[-1] for f in format_path]
            for format in formats:
                
                files = glob.glob(path + f'{sex}/{year}_{sex[0]}/{location}/metadata_2_{location}_{format}.txt')                
                for file in files_loop:
                    
                    try:
                        pd.read_csv(file, sep = '\t')
                    except:
                        i += 1
                        print(file, i)
                        

In [6]:
pd.read_csv('/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Oestersund/metadata_Oestersund_sprint.txt', sep = '\t', index_col = 0)

,30 min.\nbefore Start,at\nStart Time,30 min.\nafter Start,at\nEnd Time
Sky,Mostly Cloudy,Mostly Cloudy,Mostly Cloudy,Mostly Cloudy
Snow Condition,Compact,Compact,Compact,Compact
Snow Temperature (in °C),-2,-2,-2,-2
Air Temperature (in °C),-6,-5.8,-5.8,-6.3
Humidity (in %),79,77,78,78
Wind,SE / 0.3 m/s,SW / 0.4 m/s,S / 1.0 m/s,SW / 0.9 m/s


## Creation of the dataframe

In [7]:
re.findall('[0-9][0-9]*\.[0-9]', '/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Oberhof/intermediates/mass_start/13.5km.txt')

['13.5']

In [8]:
input_predictor = pd.DataFrame(columns=['Rank',
 'Bib',
 'Family\xa0Name',
 'Given Name',
 'Nation',
 'Sector Time',
 'Sector Rank',
 'Net Time',
 'Behind']) #Creating the DataFrame to host the input data

In [9]:
sexes

['men', 'women']

In [3]:
def find_distance(file):
    return float(re.findall('[0-9][0-9]*\.[0-9]', file)[0])

In [4]:
def to_seconds(string):
    string = str(string)
    splitted_string =  string.split(':')
    if len(splitted_string) == 1:
        return float(splitted_string[0])
    elif len(splitted_string) == 2:
        return float(splitted_string[0])*60 + float(splitted_string[1])

In [12]:
indexes_race = {'men':int(0), 'women':int(0)}
pattern = '[0-9][0-9]*\.[0-9]'


for year in years:
    for sex in sexes:
        locations = os.listdir(path + f'{sex}/{year}_{sex[0]}')
        locations = np.array(locations)[np.array(locations) !=  '.DS_Store']
        for location in locations:
            format_path = glob.glob((path + f'{sex}/{year}_{sex[0]}/{location}/intermediates/*'))
            formats = [f.split('/')[-1] for f in format_path]
            for format in formats:
                indexes_race[sex] += 1        
                files = glob.glob(path + f'{sex}/{year}_{sex[0]}/{location}/intermediates/{format}/*.txt')
                
                metadata_2 = pd.read_csv(path + f'{sex}/{year}_{sex[0]}/{location}/metadata_2_{location}_{format}.txt', sep = '\t', index_col = 0)
                
                    
                    
                metadata_2 = metadata_2.T
                distances = list(map(find_distance, files))
                indexes = np.argsort(np.array(distances))
                files = np.array(files)[indexes]
                for file in files:
                    with open(file, mode = 'r') as f_0:
                        if f_0.readlines()[1] == 'Behind\n':
                            with open(file, mode = 'r') as f:
                        
                                lines = [line for line in f.readlines()]
                                if lines[1] == 'Behind\n':
                                    with open(file, mode = 'w') as f_2:
                                        line_1 = lines[0][:-1] + 'Behind\n'
                                        f_2.write(line_1 + reduce(lambda x,y: x + y, lines[2:]))
                    
                    #display(metadata_2)
                    df = pd.read_csv(file, sep='\t')
                    distance = re.findall(pattern, file)[0]
                    df['sex'] = np.array([sex]*len(df))
                    df['distance'] = np.array([distance]*len(df))
                    df['race'] = np.array([indexes_race[sex]]*len(df))
                    df['source'] = np.array(['intermediate']*len(df))
                    df['format'] = np.array([format]*len(df))
                    df['season'] = np.array([year]*len(df))
                    
                    df['date'] = metadata_2['Date'][0]

                    
                    
                    try:
                        df['Behind'][df['Behind'] == '+2:43.3zy'] = '+2:43.3'
                    except:
                        print('WARNINGGGGGG', file) 

                    df['Behind'] = np.array((map(to_seconds, df['Behind'])))
                
                    idx_sort_behind = df['Behind'].argsort().argsort()
                

                    df['position'] = idx_sort_behind + 1
                    df['venue'] = np.array([location]*len(df))
                

                
                    input_predictor = pd.concat([input_predictor, df], axis = 0)
                    

/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_2873/2831751344.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Behind'][df['Behind'] == '+2:43.3zy'] = '+2:43.3'
/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_2873/2831751344.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Behind'][df['Behind'] == '+2:43.3zy'] = '+2:43.3'
/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_2873/2831751344.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [13]:
#input_predictor.to_csv('predictor_df.csv', index=False)

## Arrichment of the dataframe

In [14]:


with open('/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Kontiolahti/intermediates/pursuit/3.6km.txt', mode = 'r') as f:
    
    lines = [line for line in f.readlines()]
    if lines[1] == 'Behind\n':
        with open('/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Kontiolahti/intermediates/pursuit/3.6km.txt', mode = 'w') as f_2:
            line_1 = lines[0][:-1] + 'Behind\n'
            f_2.write(line_1 + reduce(lambda x,y: x + y, lines[2:]))

In [5]:
loops_race = {'sprint': 2, 'pursuit': 4, 'mass_start': 4, 'individual': 4, 'sprint_2': 2, 'pursuit_2': 4}
race_length = {'men' : {'sprint': 10, 'pursuit': 12.5, 'mass_start': 15, 'individual': 20,\
    'sprint_2': 10, 'pursuit_2':12.5}, 'women' : {'sprint': 7.5, 'pursuit': 10, 'mass_start': 12.5, 'individual': 15,\
        'sprint_2': 7.5, 'pursuit_2':10}}

In [ ]:
with open('/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2014-2015_m/Anterselva/loop_3_Anterselva_sprint.txt', 'r') as f:
    columns_loop = f.readlines()[0]
    
columns_loop = columns_loop.split('\t')
columns_loop

['Rank',
 'Bib',
 'Family\xa0Name',
 'Given Name',
 'Nation',
 'Cumulative Time',
 'Loop Time',
 'Course Time\n']

In [6]:
def to_seconds_loop(string):
    string = str(string)
    if '+' in string:
        return to_seconds(string)
    else:
        return 0

In [241]:
loops_race

{'sprint': 2,
 'pursuit': 4,
 'mass_start': 4,
 'individual': 4,
 'sprint_2': 2,
 'pursuit_2': 4}

In [297]:
indexes_race = {'men':int(0), 'women':int(0)}
df_loops = pd.DataFrame(columns = columns_loop)

for year in years:
    for sex in sexes:
        locations = os.listdir(path + f'{sex}/{year}_{sex[0]}')
        locations = np.array(locations)[np.array(locations) !=  '.DS_Store']
        for location in locations:
            format_path = glob.glob((path + f'{sex}/{year}_{sex[0]}/{location}/intermediates/*'))
            formats = [f.split('/')[-1] for f in format_path]
            for format in formats:
                indexes_race[sex] += 1        
                for loop_number in range(loops_race[format]):
                    file = path + f'{sex}/{year}_{sex[0]}/{location}/loop_{loop_number + 1}_{location}_{format}.txt'
                    
                    df = pd.read_csv(file, sep = '\t')
                    metadata_2 = pd.read_csv(path + f'{sex}/{year}_{sex[0]}/{location}/metadata_2_{location}_{format}.txt', sep = '\t', index_col = 0)
                    metadata_2 = metadata_2.T

                    df['distance'] = [(race_length[sex][format]/(loops_race[format] + 1))*(loop_number + 1)]*len(df)
                    df['sex'] = [sex]*len(df)
                    df['date'] = metadata_2['Date'][0]
                    df['race'] = np.array([indexes_race[sex]]*len(df))
                    df['source'] = np.array(['loop']*len(df))
                    df['format'] = np.array([format]*len(df))

                    df['Behind'] = np.array((map(to_seconds_loop, df['Cumulative Time'])))
                    df['season'] = np.array([year]*len(df))
                    df['venue'] = np.array([location]*len(df))
                
                    idx_sort_behind = df['Behind'].argsort().argsort()
                

                    df['position'] = idx_sort_behind + 1
                
                    df_loops = pd.concat([df_loops, df], axis = 0)
                    

In [298]:
#df_loops.to_csv('loops_df.csv', index=False)

#--------------------------------------------#

<center>
<img src="start_here.jpeg" alt="">
</center>

In [7]:
df_predictor = pd.read_csv('predictor_df.csv')
df_predictor = df_predictor.astype({'race': 'int32'})
df_predictor.head()

/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_2194/3464265586.py:1: DtypeWarning: Columns (0,1,6,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_predictor = pd.read_csv('predictor_df.csv')


,Rank,Bib,Family Name,Given Name,Nation,Sector Time,Sector Rank,Net Time,Behind,sex,...,race,source,format,season,date,position,venue,vRank,Unnamed: 0,Day Time
0,1,3,Schempp,Simon,GER,2:10.1,4.0,2:10.1,1.2,men,...,1,intermediate,mass_start,2016-2017,2017-01-08,4.0,Oberhof,NaN,NaN,NaN
1,2,6,Lesser,Erik,GER,2:11.0,7.0,2:11.0,2.1,men,...,1,intermediate,mass_start,2016-2017,2017-01-08,7.0,Oberhof,NaN,NaN,NaN
2,3,1,Fourcade,Martin,FRA,2:08.9,1.0,2:08.9,0.0,men,...,1,intermediate,mass_start,2016-2017,2017-01-08,1.0,Oberhof,NaN,NaN,NaN
3,4,16,Beatrix,Jean Guillaume,FRA,2:15.5,23.0,2:15.5,6.6,men,...,1,intermediate,mass_start,2016-2017,2017-01-08,22.0,Oberhof,NaN,NaN,NaN
4,5,8,Bjoerndalen,Ole Einar,NOR,2:11.3,8.0,2:11.3,2.4,men,...,1,intermediate,mass_start,2016-2017,2017-01-08,8.0,Oberhof,NaN,NaN,NaN


In [8]:
df_loops = pd.read_csv('loops_df.csv')

In [9]:
df_loops.reset_index(inplace = True, drop = True)
df_loops['race'] = df_loops['race'].astype({'race': 'int32'})

df_loops.head() 

,Rank,Bib,Family Name,Given Name,Nation,Cumulative Time,Loop Time,Course Time\n,Course Time,Range Time,...,sex,date,race,source,format,Behind,season,venue,position,Ski Time
0,1,3,Schempp,Simon,GER,+1.5,+1.5,NaN,+2.1,8.3,...,men,2017-01-08,1,loop,mass_start,1.5,2016-2017,Oberhof,6.0,NaN
1,2,6,Lesser,Erik,GER,+0.1,+0.1,NaN,+4.4,4.2,...,men,2017-01-08,1,loop,mass_start,0.1,2016-2017,Oberhof,2.0,NaN
2,3,1,Fourcade,Martin,FRA,8:28.7,8:28.7,NaN,+0.7,7.6,...,men,2017-01-08,1,loop,mass_start,0.0,2016-2017,Oberhof,1.0,NaN
3,4,16,Beatrix,Jean Guillaume,FRA,+2.8,+2.8,NaN,+6.0,5.9,...,men,2017-01-08,1,loop,mass_start,2.8,2016-2017,Oberhof,8.0,NaN
4,5,8,Bjoerndalen,Ole Einar,NOR,+1.2,+1.2,NaN,+4.7,5.5,...,men,2017-01-08,1,loop,mass_start,1.2,2016-2017,Oberhof,5.0,NaN


## Building predictor's entries

In [10]:
df_predictor['km_to_go'] = df_predictor.apply(lambda x: race_length[x.sex][x.format] - x.distance, axis = 1)
df_loops['km_to_go'] = df_loops.apply(lambda x: race_length[x.sex][x.format] - x.distance, axis = 1)

In [11]:
df_predictor['date'] = list(map(lambda x: x.strip(), df_predictor['date']))

In [12]:
all_dates = df_predictor['date'].unique()
all_dates_datetime = pd.to_datetime(all_dates, format = '%Y-%m-%d')

In [13]:
idx_dates = np.array(all_dates_datetime).argsort().argsort() + 1

dict_date_idx = {all_dates[i]: idx_dates[i] for i in range(len(all_dates))}

In [14]:
df_predictor['date_order'] = df_predictor['date'].map(dict_date_idx)
df_loops['date_order'] = df_loops['date'].map(dict_date_idx)

In [15]:
format_to_ranges ={'individual': ['prone', 'standing', 'prone', 'standing'], 'sprint': ['prone', 'standing'], 'pursuit': ['prone', 'prone', 'standing', 'standing'],\
    'mass_start' : ['prone', 'prone', 'standing', 'standing'], 'sprint_2': ['prone', 'standing'], 'pursuit_2': ['prone', 'prone', 'standing', 'standing']}

In [16]:
race_length

{'men': {'sprint': 10,
  'pursuit': 12.5,
  'mass_start': 15,
  'individual': 20,
  'sprint_2': 10,
  'pursuit_2': 12.5},
 'women': {'sprint': 7.5,
  'pursuit': 10,
  'mass_start': 12.5,
  'individual': 15,
  'sprint_2': 7.5,
  'pursuit_2': 10}}

In [16]:
def km_to_ranges(km, format, sex):
    tot_km = race_length[sex][format]
    km_per_loop = tot_km/(loops_race[format] + 1)
    shooting_ranges_km = [km_per_loop * i for i in np.arange(loops_race[format]) + 1]

    range_done = sum(km >= np.array(shooting_ranges_km) - 0.05)
    
    n_standing = format_to_ranges[format][range_done:].count('standing')
    n_prone = format_to_ranges[format][range_done:].count('prone')

    return (n_standing, n_prone)


In [17]:
df_predictor[['standing_remaining', 'prone_remaining']] = df_predictor.apply(lambda x: km_to_ranges(x.distance, x.format, x.sex), axis = 1, result_type = 'expand')

In [18]:
df_loops[['standing_remaining', 'prone_remaining']] = df_loops.apply(lambda x: km_to_ranges(x.distance, x.format, x.sex), axis = 1, result_type = 'expand')

In [21]:
df_predictor.columns = ['Rank', 'Bib', 'Family Name', 'Given Name', 'Nation', 'Sector Time',
       'Sector Rank', 'Net Time', 'Behind', 'sex', 'distance', 'race',
       'source', 'format', 'season', 'date', 'position', 'venue', 'vRank',
       'Unnamed: 0', 'Day Time', 'km_to_go', 'date_order',
       'standing_remaining', 'prone_remaining']

In [22]:
df_predictor['full_name'] = df_predictor['Given Name'] + '_' + df_predictor['Family Name']

In [23]:
df_loops.columns = ['Rank', 'Bib', 'Family Name', 'Given Name', 'Nation', 'Cumulative Time',
       'Loop Time', 'Course Time\n', 'Course Time', 'Range Time',
       'Shooting Time', 'Penalty Time', 'distance', 'sex', 'date', 'race',
       'source', 'format', 'Behind', 'season', 'venue', 'position', 'Ski Time',
       'km_to_go', 'date_order', 'standing_remaining', 'prone_remaining']

In [24]:
df_loops['full_name'] = df_loops['Given Name'] + '_' + df_loops['Family Name']

In [25]:
df_predictor['full_name'][df_predictor['full_name'] == 'Alexandr_Loginov'] = 'Alexander_Loginov'

/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_2194/2715052277.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictor['full_name'][df_predictor['full_name'] == 'Alexandr_Loginov'] = 'Alexander_Loginov'


In [26]:
df_loops['full_name'][df_loops['full_name'] == 'Alexandr_Loginov'] = 'Alexander_Loginov'

/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_2194/1105425695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loops['full_name'][df_loops['full_name'] == 'Alexandr_Loginov'] = 'Alexander_Loginov'


In [26]:
df_loops

,Rank,Bib,Family Name,Given Name,Nation,Cumulative Time,Loop Time,Course Time\n,Course Time,Range Time,...,Behind,season,venue,position,Ski Time,km_to_go,date_order,standing_remaining,prone_remaining,full_name
0,1,3,Schempp,Simon,GER,+1.5,+1.5,NaN,+2.1,8.3,...,1.5,2016-2017,Oberhof,6.0,NaN,12.0,14.0,2,1,Simon_Schempp
1,2,6,Lesser,Erik,GER,+0.1,+0.1,NaN,+4.4,4.2,...,0.1,2016-2017,Oberhof,2.0,NaN,12.0,14.0,2,1,Erik_Lesser
2,3,1,Fourcade,Martin,FRA,8:28.7,8:28.7,NaN,+0.7,7.6,...,0.0,2016-2017,Oberhof,1.0,NaN,12.0,14.0,2,1,Martin_Fourcade
3,4,16,Beatrix,Jean Guillaume,FRA,+2.8,+2.8,NaN,+6.0,5.9,...,2.8,2016-2017,Oberhof,8.0,NaN,12.0,14.0,2,1,Jean Guillaume_Beatrix
4,5,8,Bjoerndalen,Ole Einar,NOR,+1.2,+1.2,NaN,+4.7,5.5,...,1.2,2016-2017,Oberhof,5.0,NaN,12.0,14.0,2,1,Ole Einar_Bjoerndalen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63585,71,78,Sabule,Annija,LAT,+2:40.7,+1:43.9,NaN,+1:18.4,10.0,...,160.7,2021-2022,Oslo_Holmenkollen,62.0,NaN,2.5,202.0,0,0,Annija_Sabule
63586,72,66,Colebourn,Jillian Wei-Lin,AUS,+3:35.1,+2:28.1,NaN,+1:34.6,10.3,...,215.1,2021-2022,Oslo_Holmenkollen,72.0,NaN,2.5,202.0,0,0,Jillian Wei-Lin_Colebourn
63587,73,64,Bulina,Sanita,LAT,+3:36.1,+2:18.0,NaN,+1:26.0,7.4,...,216.1,2021-2022,Oslo_Holmenkollen,73.0,NaN,2.5,202.0,0,0,Sanita_Bulina
63588,74,50,Tanaka,Yurie,JPN,+4:14.8,+1:00.7,NaN,+59.0,15.6,...,254.8,2021-2022,Oslo_Holmenkollen,75.0,NaN,2.5,202.0,0,0,Yurie_Tanaka


# Percentage standing and percentage shooting

In [27]:
df_loops['shootings_done'] = df_loops.apply(lambda x: len(format_to_ranges[x.format]) - x.standing_remaining - x.prone_remaining, axis = 1)

In [28]:
format_to_ranges

{'individual': ['prone', 'standing', 'prone', 'standing'],
 'sprint': ['prone', 'standing'],
 'pursuit': ['prone', 'prone', 'standing', 'standing'],
 'mass_start': ['prone', 'prone', 'standing', 'standing'],
 'sprint_2': ['prone', 'standing'],
 'pursuit_2': ['prone', 'prone', 'standing', 'standing']}

In [29]:
df_loops[df_loops['shootings_done'] == 0].format.unique()

array([], dtype=object)

In [30]:
unique_names_loop = df_loops['full_name'].unique()

In [ ]:
problems_file = []

In [63]:
second_df_loop = pd.DataFrame(columns = df_loops.columns)


unique_names_loop = df_loops['full_name'].unique()


for name in unique_names_loop:
    

    tmp = df_loops[df_loops['full_name'] == name]
    seasons = tmp['season'].unique()
    for season in seasons:
        percentages_back = []
        probabilities_standing = [0]
        probabilities_prone = [0]

        mistakes_standing = [0]
        mistakes_prone = [0]

        #mistakes_standing = [0]
        #mistakes_prone = [0]
        tmp_2 = tmp[tmp['season'] == season]

        tmp_2.sort_values(by = ['date_order', 'distance'], ascending = True, inplace = True)
        
        #mistakes_dict = {'standing': [0], 'prone': [0]}
        hits = {'standing': [0], 'prone': [0]}
        tryes = {'standing': [0], 'prone': [0]}
        #display(tmp_2.iloc[:,:18])
        for i in range(len(tmp_2)):
            date_current = tmp_2.iloc[i, :].date_order
            if i >= 1:
                date_previous = tmp_2.iloc[i-1, :].date_order
            else:
                date_previous = date_current - 1
            #print(f'/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/{tmp_2.iloc[i, :].sex}/{tmp_2.iloc[i, :].season}_{tmp_2.iloc[i, :].sex}/{tmp_2.iloc[i, :].venue}/shooting_{tmp_2.iloc[i, :].venue}_{tmp_2.iloc[i, :].format}.txt')
            
            try:
                #print(f'/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/{tmp_2.iloc[i, :].sex}/{tmp_2.iloc[i, :].season}_{tmp_2.iloc[i, :].sex[0]}/{tmp_2.iloc[i, :].venue}/shooting_{tmp_2.iloc[i, :].venue}_{tmp_2.iloc[i, :].format}.txt')
                df = pd.read_csv(f'/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/{tmp_2.iloc[i, :].sex}/{tmp_2.iloc[i, :].season}_{tmp_2.iloc[i, :].sex[0]}/{tmp_2.iloc[i, :].venue}/shooting_{tmp_2.iloc[i, :].venue}_{tmp_2.iloc[i, :].format}.txt', sep = '\t')
                
            
                
                #print(f'/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/{tmp_2.iloc[i, :].sex}/{tmp_2.iloc[i, :].season}_{tmp_2.iloc[i, :].sex[0]}/{tmp_2.iloc[i, :].venue}/analysis_{tmp_2.iloc[i, :].venue}_{tmp_2.iloc[i, :].format}.txt')   
                df_analysis = pd.read_csv(f'/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/{tmp_2.iloc[i, :].sex}/{tmp_2.iloc[i, :].season}_{tmp_2.iloc[i, :].sex[0]}/{tmp_2.iloc[i, :].venue}/analysis_{tmp_2.iloc[i, :].venue}_{tmp_2.iloc[i, :].format}.txt', sep = '\t')
                df_analysis['full_name'] = df_analysis['Given Name'] + '_' + df_analysis['Family\xa0Name']
                percentage_back = df_analysis[df_analysis['full_name'] == name]
                
                percentage_back_value = percentage_back['back from\nmedian\n(in %)'].values[0]


                
                percentages_back.append(percentage_back_value)
                    
                

                df['full_name'] = df['Given Name'] + '_' + df['Family\xa0Name']
                df['full_name'][df['full_name'] == 'Alexandr_Loginov'] = 'Alexander_Loginov'

                entry = df[df['full_name'] == tmp_2.iloc[i, :]['full_name']]
                shooting_string = entry[f'Shooting {tmp_2.iloc[i, :].shootings_done}']
                

                shooting_string = str(shooting_string).split('\\n')[1][:5]
                
                #mistakes = shooting_string.count('0')
                hits_ = 5 - shooting_string.count('0')
                
                mistakes = shooting_string.count('0')


                #mistakes_dict[format_to_ranges[str(tmp_2.iloc[i, :].format)][tmp_2.iloc[i, :].shootings_done - 1]].append(mistakes)

                hits[format_to_ranges[str(tmp_2.iloc[i, :].format)][tmp_2.iloc[i, :].shootings_done - 1]].\
                    append(hits[format_to_ranges[str(tmp_2.iloc[i, :].format)][tmp_2.iloc[i, :].shootings_done - 1]][-1] + hits_)
                
                tryes[format_to_ranges[str(tmp_2.iloc[i, :].format)][tmp_2.iloc[i, :].shootings_done - 1]].\
                    append(tryes[format_to_ranges[str(tmp_2.iloc[i, :].format)][tmp_2.iloc[i, :].shootings_done - 1]][-1] + 5)

                #if format_to_ranges[str(tmp_2.iloc[i, :].format)][tmp_2.iloc[i, :].shootings_done - 1] == 'standing':
                 #   mistakes_standing.append(mistakes_standing[-1] + mistakes)
                  #  mistakes_prone.append(mistakes_prone[-1])

                #elif format_to_ranges[str(tmp_2.iloc[i, :].format)][tmp_2.iloc[i, :].shootings_done - 1] == 'prone':
                 #   mistakes_prone.append(mistakes_prone[-1] + mistakes)
                  #  mistakes_standing.append(mistakes_standing[-1])
                    
                if format_to_ranges[str(tmp_2.iloc[i, :].format)][tmp_2.iloc[i, :].shootings_done - 1] == 'standing':
                    if date_current == date_previous:
                        mistakes_standing.append(mistakes_standing[-1] + mistakes)
                        mistakes_prone.append(mistakes_prone[-1])
                    else:
                        mistakes_standing.append(mistakes)
                        mistakes_prone.append(0)
                
                if format_to_ranges[str(tmp_2.iloc[i, :].format)][tmp_2.iloc[i, :].shootings_done - 1] == 'prone':
                    if date_current == date_previous:
                        mistakes_prone.append(mistakes_prone[-1] + mistakes)
                        mistakes_standing.append(mistakes_standing[-1])
                    else:
                        mistakes_prone.append(mistakes)
                        mistakes_standing.append(0)
                    
                if  tryes['standing'][-1] != 0:
                    probabilities_standing.append(hits['standing'][-1] / tryes['standing'][-1])
                else:
                    probabilities_standing.append(0)
                
                if tryes['prone'][-1] != 0:
                    probabilities_prone.append(hits['prone'][-1] / tryes['prone'][-1])
                else:
                    probabilities_prone.append(0)


            except (FileNotFoundError, IndexError):
                #print(f'/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/{tmp_2.iloc[i, :].sex}/{tmp_2.iloc[i, :].season}_{tmp_2.iloc[i, :].sex[0]}/{tmp_2.iloc[i, :].venue}/shooting_{tmp_2.iloc[i, :].venue}_{tmp_2.iloc[i, :].format}.txt')
                #mistakes_standing.append(0) # Be aware, the mistakes will not 100% reflect the real probailities because of this append. 
                #mistakes_prone.append(0)
                mistakes_standing.append(mistakes_standing[-1])
                mistakes_prone.append(mistakes_prone[-1])
                probabilities_standing.append(probabilities_standing[-1])
                probabilities_prone.append(probabilities_prone[-1])
                percentages_back.append(np.nan)

            #if name == 'Matvey_Eliseev':
            
            if len(percentages_back) != len(mistakes_prone[1:]): 
                percentages_back = percentages_back[:-1]
                

        #probabilities_standing = np.array(hits['standing'][1:]) / np.array(tryes['standing'][1:])
        #probabilities_prone = np.array(hits['prone'][1:]) / np.array(tryes['prone'][1:])
        #tmp_2['mistakes_standing'] = mistakes_standing[1:]
        #tmp_2['mistakes_prone'] = mistakes_prone[1:]
        
        tmp_2['probability_standing'] = probabilities_standing[1:]
        tmp_2['probability_prone'] = probabilities_prone[1:]
        tmp_2['all_mistakes_prone'] = mistakes_prone[1:]
        tmp_2['all_mistakes_standing'] = mistakes_standing[1:]
        
            
        tmp_2['percentage_back'] = percentages_back
        
        second_df_loop = pd.concat([second_df_loop, tmp_2])



In [64]:
second_df_loop

,Rank,Bib,Family Name,Given Name,Nation,Cumulative Time,Loop Time,Course Time\n,Course Time,Range Time,...,date_order,standing_remaining,prone_remaining,full_name,shootings_done,probability_standing,probability_prone,all_mistakes_prone,all_mistakes_standing,percentage_back
1585,46,9,Schempp,Simon,GER,+18.4,+18.4,NaN,+10.1,10.9,...,2.0,2,1,Simon_Schempp,1,0.0,1.000000,0.0,0.0,-5.06
1688,46,9,Schempp,Simon,GER,+4:48.6,+4:36.0,NaN,+9.4,32.6,...,2.0,1,1,Simon_Schempp,2,0.2,1.000000,0.0,4.0,-5.06
1791,46,9,Schempp,Simon,GER,+5:44.3,+1:03.4,NaN,+3.2,9.4,...,2.0,1,0,Simon_Schempp,3,0.2,0.900000,1.0,4.0,-5.06
1894,46,9,Schempp,Simon,GER,+5:37.6,+2:03.7,NaN,9:28.3,+9.7,...,2.0,0,0,Simon_Schempp,4,0.4,0.900000,1.0,6.0,-5.06
1350,21,22,Schempp,Simon,GER,+36.7,+36.7,NaN,+7.7,11.2,...,3.0,1,0,Simon_Schempp,1,0.4,0.866667,1.0,0.0,-3.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62673,85,73,Horodna,Olena,UKR,+4:04.3,+2:36.0,NaN,+1:15.8,7.6,...,200.0,0,0,Olena_Horodna,2,0.4,0.800000,1.0,3.0,10.96
62984,99,110,Arnekleiv,Juni,NOR,+41.2,+41.2,NaN,+36.7,7.2,...,184.0,1,0,Juni_Arnekleiv,1,0.0,1.000000,0.0,0.0,7.56
63095,99,110,Arnekleiv,Juni,NOR,+2:11.8,+1:30.6,NaN,+1:02.0,10.1,...,184.0,0,0,Juni_Arnekleiv,2,0.8,1.000000,0.0,1.0,7.56
62991,106,106,Dimitrova,Valentina,BUL,+1:07.4,+1:07.4,NaN,+1:03.1,7.7,...,184.0,1,0,Valentina_Dimitrova,1,0.0,1.000000,0.0,0.0,11.66


In [47]:
unique_names_loop = df_loops['full_name'].unique()
all_back_from_medians = []
for name in unique_names_loop:
    df_names = second_df_loop[second_df_loop['full_name'] == name]
    name_back = []
    seasons = df_names['season'].unique()
    for season in seasons:

        back_from_median_final = []

        uniques_back_from_median = []

        df_season = df_names[df_names['season'] == season]


        #date_orders = df_season['date_order'].unique()
        

        for i in range(len(df_season)):
            current_date = df_season.iloc[i, :].date_order
            date = df_season.iloc[i, :].date
            date_df = df_season[df_season['date'] == date]
            percentage_back = np.mean(date_df['percentage_back'][~ np.isnan(date_df['percentage_back'])])

            if np.isnan(percentage_back):
                if not np.isnan(np.mean(np.array(name_back)[~ np.isnan(np.array(name_back))])):
                    percentage_back = np.mean(np.array(name_back)[~ np.isnan(np.array(name_back))])
                else:
                    print('yes')
                    percentage_back = 0

            if i != 0:
                previous_date = df_season.iloc[i - 1, :].date_order
            else:
                previous_date = -1

            if current_date != previous_date:
                
                uniques_back_from_median.append(percentage_back)
                back_from_median_final.append(np.mean(uniques_back_from_median))
            else:
                back_from_median_final.append(back_from_median_final[-1])
            name_back.append(percentage_back)
        
        all_back_from_medians += back_from_median_final
            


yes
yes


In [48]:
#second_df_loop['back_from_median'] = all_back_from_medians

In [49]:
#second_df_loop.to_csv('second_df_loop.csv', index = False)

In [31]:
second_df_loop = pd.read_csv('second_df_loop.csv')

In [82]:
#df_predictor.to_csv('df_predictor.csv', index = False)

In [32]:
df_predictor = pd.read_csv('df_predictor.csv')

In [33]:
df_predictor.columns

Index(['Rank', 'Bib', 'Family Name', 'Given Name', 'Nation', 'Sector Time',
       'Sector Rank', 'Net Time', 'Behind', 'sex', 'distance', 'race',
       'source', 'format', 'season', 'date', 'position', 'venue', 'vRank',
       'Unnamed: 0', 'Day Time', 'km_to_go', 'date_order',
       'standing_remaining', 'prone_remaining', 'full_name'],
      dtype='object')

In [34]:
df_input_pred = df_predictor[['full_name', 'distance', 'Rank', 'sex', 'Behind', 'format', 'season', 'date', 'date_order', 'position'\
    , 'km_to_go', 'standing_remaining', 'prone_remaining']]

In [35]:
second_df_loop.columns

Index(['Rank', 'Bib', 'Family Name', 'Given Name', 'Nation', 'Cumulative Time',
       'Loop Time', 'Course Time\n', 'Course Time', 'Range Time',
       'Shooting Time', 'Penalty Time', 'distance', 'sex', 'date', 'race',
       'source', 'format', 'Behind', 'season', 'venue', 'position', 'Ski Time',
       'km_to_go', 'date_order', 'standing_remaining', 'prone_remaining',
       'full_name', 'shootings_done', 'probability_standing',
       'probability_prone', 'all_mistakes_prone', 'all_mistakes_standing',
       'percentage_back', 'back_from_median'],
      dtype='object')

In [36]:
df_input_loop = second_df_loop[['full_name', 'distance', 'Rank', 'sex', 'Behind', 'format', 'season', 'date', 'date_order', 'position'\
    , 'km_to_go', 'standing_remaining', 'prone_remaining', 'probability_standing', 'probability_prone',\
         'all_mistakes_prone', 'all_mistakes_standing', 'back_from_median']]

In [37]:
df_input = pd.concat([df_input_pred, df_input_loop])

In [38]:
df_input.sort_values(by = ['season', 'date', 'full_name', 'distance'], ascending=True, inplace = True)
df_input.reset_index(inplace = True, drop = True)

In [76]:
second_df_input = pd.DataFrame(columns = df_input.columns)
for name in df_input['full_name'].unique():
    df_name = df_input[df_input['full_name'] == name]
    
    for season in df_name['season'].unique():
        df_season = df_name[df_name['season'] == season]
        
        

        for date in df_season['date'].unique():
            df_date = df_season[df_season['date'] == date]
            
            df_date.fillna(method = 'ffill', inplace = True)
            
            second_df_input = pd.concat([second_df_input, df_date])


        
#df_input.fillna(method = 'ffill', inplace = True)

In [77]:
third_df_input = pd.DataFrame(columns = second_df_input.columns)
for name in second_df_input['full_name'].unique():
    df_name = second_df_input[second_df_input['full_name'] == name]
    
    for season in df_name['season'].unique():
        df_season = df_name[df_name['season'] == season]
        df_season[['probability_standing', 'probability_prone']] = df_season[['probability_standing', 'probability_prone']]\
            .fillna(df_season[['probability_standing', 'probability_prone']].mean())

        df_season[['all_mistakes_prone', 'all_mistakes_standing']] = df_season[['all_mistakes_prone', 'all_mistakes_standing']]\
            .fillna(value = 0)

        df_season['back_from_median'] = df_season['back_from_median']\
            .fillna(method = 'bfill')

        third_df_input = pd.concat([third_df_input, df_season])

In [79]:
#third_df_input.to_csv('third_df_input.csv', index = False)

In [13]:
third_df_input = pd.read_csv('third_df_input.csv')

In [15]:
fourth_df_input =pd.DataFrame(columns = third_df_input.columns)
all_gradients = []
for name in third_df_input['full_name'].unique():
      df_name = third_df_input[third_df_input['full_name'] == name]
      list_final_gradient = []
      for season in df_name['season'].unique():
         df_season = df_name[df_name['season'] == season]
         dict_date = {str(date) : np.mean(df_season[df_season['date'] == date].back_from_median) for date in df_season['date'].unique()}

         dict_date_standing = {str(date) : np.mean(df_season[df_season['date'] == date].probability_standing)\
             for date in df_season['date'].unique()}
         
         dict_date_prone = {str(date) : np.mean(df_season[df_season['date'] == date].probability_prone)\
             for date in df_season['date'].unique()}

         back_from_medians = np.array(list(dict_date.values()))

         standing_probabilities = np.array(list(dict_date_standing.values()))

         prone_probabilities = np.array(list(dict_date_prone.values()))

         df_backs = pd.DataFrame(back_from_medians, columns = ['tmp'])

         df_standings = pd.DataFrame(standing_probabilities, columns = ['tmp'])

         df_prone = pd.DataFrame(prone_probabilities, columns = ['tmp'])

         list_gradients = [0]

         list_gradients_standing = [0]

         list_gradients_prone = [0]

         if len(df_backs) > 5:
            list_gradients += [df_backs['tmp'][1] - df_backs['tmp'][0], np.mean([df_backs['tmp'][2] - df_backs['tmp'][1], df_backs['tmp'][1]\
               - df_backs['tmp'][0]]),\
            np.mean([df_backs['tmp'][3] - df_backs['tmp'][2], df_backs['tmp'][2] - df_backs['tmp'][1], df_backs['tmp'][1] - df_backs['tmp'][0]]),\
            np.mean([df_backs['tmp'][4] - df_backs['tmp'][3], df_backs['tmp'][3] - df_backs['tmp'][2],\
               df_backs['tmp'][2] - df_backs['tmp'][1], df_backs['tmp'][1] - df_backs['tmp'][0]])]

            
            


            for i in range(len(df_backs) - 5):
               list_gradients.append(np.mean(np.array([df_backs['tmp'][i + j + 1] - df_backs['tmp'][i + j] for j in range(4)])))

            
         elif len(df_backs) == 5:
            list_gradients += [df_backs['tmp'][1] - df_backs['tmp'][0], np.mean([df_backs['tmp'][2] - df_backs['tmp'][1], df_backs['tmp'][1]\
               - df_backs['tmp'][0]]),\
            np.mean([df_backs['tmp'][3] - df_backs['tmp'][2], df_backs['tmp'][2] - df_backs['tmp'][1], df_backs['tmp'][1] - df_backs['tmp'][0]]),\
            np.mean([df_backs['tmp'][4] - df_backs['tmp'][3], df_backs['tmp'][3] - df_backs['tmp'][2],\
               df_backs['tmp'][2] - df_backs['tmp'][1], df_backs['tmp'][1] - df_backs['tmp'][0]])]

         elif len(df_backs) == 4:
            list_gradients += [df_backs['tmp'][1] - df_backs['tmp'][0], np.mean([df_backs['tmp'][2] - df_backs['tmp'][1], df_backs['tmp'][1]\
               - df_backs['tmp'][0]]), np.mean([df_backs['tmp'][3] - df_backs['tmp'][2], df_backs['tmp'][2] - df_backs['tmp'][1],\
                   df_backs['tmp'][1] - df_backs['tmp'][0]])]
                  
         elif len(df_backs) == 3:
            list_gradients += [df_backs['tmp'][1] - df_backs['tmp'][0], np.mean([df_backs['tmp'][2] - df_backs['tmp'][1],\
                df_backs['tmp'][1] - df_backs['tmp'][0]])]

         elif len(df_backs) == 2:
            list_gradients += [df_backs['tmp'][1] - df_backs['tmp'][0]]

         elif len(df_backs) == 1:
            list_gradients += [0]

         
         if len(df_standings) > 5:
         

            for i in range(len(df_standings) - 5):
               list_gradients_standing.append(np.mean(np.array([df_standings['tmp'][i + j + 1] - df_standings['tmp'][i + j] for j in range(4)])))

            list_gradients_standing += [df_standings['tmp'][1] - df_standings['tmp'][0], np.mean([df_standings['tmp'][2] -\
                df_standings['tmp'][1], df_standings['tmp'][1] - df_standings['tmp'][0]]),\
            np.mean([df_standings['tmp'][3] - df_standings['tmp'][2], df_standings['tmp'][2] - df_standings['tmp'][1],\
                df_standings['tmp'][1] - df_standings['tmp'][0]]),\
            np.mean([df_standings['tmp'][4] - df_standings['tmp'][3], df_standings['tmp'][3] - df_standings['tmp'][2],\
               df_standings['tmp'][2] - df_standings['tmp'][1], df_standings['tmp'][1] - df_standings['tmp'][0]])]

         elif len(df_standings) == 5:
            list_gradients_standing += [df_standings['tmp'][1] - df_standings['tmp'][0], np.mean([df_standings['tmp'][2] - df_standings['tmp'][1],\
                df_standings['tmp'][1]\
               - df_standings['tmp'][0]]),\
            np.mean([df_standings['tmp'][3] - df_standings['tmp'][2], df_standings['tmp'][2] - df_standings['tmp'][1],\
                df_standings['tmp'][1] - df_standings['tmp'][0]]),\
            np.mean([df_standings['tmp'][4] - df_standings['tmp'][3], df_standings['tmp'][3] - df_standings['tmp'][2],\
               df_standings['tmp'][2] - df_standings['tmp'][1], df_standings['tmp'][1] - df_standings['tmp'][0]])]

         elif len(df_standings) == 4:
            list_gradients_standing += [df_standings['tmp'][1] - df_standings['tmp'][0], np.mean([df_standings['tmp'][2] - df_standings['tmp'][1],\
                df_standings['tmp'][1]\
               - df_standings['tmp'][0]]), np.mean([df_standings['tmp'][3] - df_standings['tmp'][2], df_standings['tmp'][2] - df_standings['tmp'][1],\
                   df_standings['tmp'][1] - df_standings['tmp'][0]])]
         
         
         elif len(df_standings) == 3:
            list_gradients_standing += [df_standings['tmp'][1] - df_standings['tmp'][0], np.mean([df_standings['tmp'][2] - df_standings['tmp'][1],\
                df_standings['tmp'][1] - df_standings['tmp'][0]])]

         elif len(df_standings) == 2:
            list_gradients_standing += [df_standings['tmp'][1] - df_standings['tmp'][0]]

         elif len(df_standings) == 1:
            list_gradients_standing += [0]

         



         if len(df_prone) > 5:

            list_gradients_prone += [df_prone['tmp'][1] - df_prone['tmp'][0], np.mean([df_prone['tmp'][2] - df_prone['tmp'][1],\
                df_prone['tmp'][1]\
               - df_prone['tmp'][0]]),\
            np.mean([df_prone['tmp'][3] - df_prone['tmp'][2], df_prone['tmp'][2] - df_prone['tmp'][1], df_prone['tmp'][1] - df_prone['tmp'][0]]),\
            np.mean([df_prone['tmp'][4] - df_prone['tmp'][3], df_prone['tmp'][3] - df_prone['tmp'][2],\
               df_prone['tmp'][2] - df_prone['tmp'][1], df_prone['tmp'][1] - df_prone['tmp'][0]])]

            
            


            for i in range(len(df_prone) - 5):
               list_gradients_prone.append(np.mean(np.array([df_prone['tmp'][i + j + 1] - df_prone['tmp'][i + j] for j in range(4)])))

            

         elif len(df_prone) == 5:
            list_gradients_prone += [df_prone['tmp'][1] - df_prone['tmp'][0], np.mean([df_prone['tmp'][2] - df_prone['tmp'][1], df_prone['tmp'][1]\
               - df_prone['tmp'][0]]),\
            np.mean([df_prone['tmp'][3] - df_prone['tmp'][2], df_prone['tmp'][2] - df_prone['tmp'][1], df_prone['tmp'][1] - df_prone['tmp'][0]]),\
            np.mean([df_prone['tmp'][4] - df_prone['tmp'][3], df_prone['tmp'][3] - df_prone['tmp'][2],\
               df_prone['tmp'][2] - df_prone['tmp'][1], df_prone['tmp'][1] - df_prone['tmp'][0]])]

         elif len(df_prone) == 4:
            list_gradients_prone += [df_prone['tmp'][1] - df_prone['tmp'][0], np.mean([df_prone['tmp'][2] - df_prone['tmp'][1], df_prone['tmp'][1]\
               - df_prone['tmp'][0]]), np.mean([df_prone['tmp'][3] - df_prone['tmp'][2], df_prone['tmp'][2] - df_prone['tmp'][1],\
                   df_prone['tmp'][1] - df_prone['tmp'][0]])]
                  
         elif len(df_prone) == 3:
            list_gradients_prone += [df_prone['tmp'][1] - df_prone['tmp'][0], np.mean([df_prone['tmp'][2] - df_prone['tmp'][1],\
                df_prone['tmp'][1] - df_prone['tmp'][0]])]

         elif len(df_prone) == 2:
            list_gradients_prone += [df_prone['tmp'][1] - df_prone['tmp'][0]]

         elif len(df_prone) == 1:
            list_gradients_prone += [0]






         dict_mapping = {list(dict_date.keys())[i] : list_gradients[i] for i in range(len(dict_date))}

         dict_mapping_standing = {list(dict_date_standing.keys())[i] : list_gradients_standing[i] for i in range(len(dict_date_standing))}

         dict_mapping_prone = {list(dict_date_prone.keys())[i] : list_gradients_prone[i] for i in range(len(dict_date_prone))}

         df_season['gradient_back'] = df_season['date'].map(dict_mapping)

         df_season['gradient_standing'] = df_season['date'].map(dict_mapping_standing)

         df_season['gradient_prone'] = df_season['date'].map(dict_mapping_prone)

         fourth_df_input = pd.concat([fourth_df_input, df_season])





In [17]:
#fourth_df_input.to_csv('fourth_df_input.csv', index = False)

In [21]:
fourth_df_input = pd.read_csv('fourth_df_input.csv')

In [24]:
fifth_df_input = pd.DataFrame(columns = fourth_df_input.columns)

for name in fourth_df_input['full_name'].unique():
      df_name = fourth_df_input[fourth_df_input['full_name'] == name]
      
      for season in df_name['season'].unique():
         df_season = df_name[df_name['season'] == season]
         all_gradient_pos_date = []
         for date in df_season['date'].unique():
            df_date = df_season[df_season['date'] == date]

            list_gradients_pos = [0]

            positions = np.array(df_date['position'])

            df_pos = pd.DataFrame({'tmp' : positions})

            if len(df_pos) > 4:
               list_gradients_pos += [df_pos['tmp'][1] - df_pos['tmp'][0], np.mean([df_pos['tmp'][2] - df_pos['tmp'][1], df_pos['tmp'][1]\
                  - df_pos['tmp'][0]]),\
               np.mean([df_pos['tmp'][3] - df_pos['tmp'][2], df_pos['tmp'][2] - df_pos['tmp'][1], df_pos['tmp'][1] - df_pos['tmp'][0]])]

               for i in range(len(df_pos) - 4):
                  list_gradients_pos.append(np.mean(np.array([df_pos['tmp'][i + j + 1] - df_pos['tmp'][i + j] for j in range(3)])))



            elif len(df_pos) == 4:
               list_gradients_pos += [df_pos['tmp'][1] - df_pos['tmp'][0], np.mean([df_pos['tmp'][2] - df_pos['tmp'][1], df_pos['tmp'][1]\
                  - df_pos['tmp'][0]]), np.mean([df_pos['tmp'][3] - df_pos['tmp'][2], df_pos['tmp'][2] - df_pos['tmp'][1],\
                     df_pos['tmp'][1] - df_pos['tmp'][0]])]
                  
            elif len(df_pos) == 3:
               list_gradients_pos += [df_pos['tmp'][1] - df_pos['tmp'][0], np.mean([df_pos['tmp'][2] - df_pos['tmp'][1],\
                  df_pos['tmp'][1] - df_pos['tmp'][0]])]

            elif len(df_pos) == 2:
               list_gradients_pos += [df_pos['tmp'][1] - df_pos['tmp'][0]]

            elif len(df_prone) == 1:
               list_gradients_pos += [0]


            all_gradient_pos_date += list_gradients_pos
         df_season['gradient_pos'] = all_gradient_pos_date

         fifth_df_input = pd.concat([fifth_df_input, df_season])



In [28]:
#fifth_df_input.to_csv('fifth_df_input.csv', index = False)

In [50]:
fifth_df_input = pd.read_csv('fifth_df_input.csv')

In [51]:
fifth_df_input.sort_values(by = ['season', 'date', 'distance', 'position'], inplace = True)
fifth_df_input.columns
#fifth_df_input[['Behind', 'full_name']]

Index(['full_name', 'distance', 'Rank', 'sex', 'Behind', 'format', 'season',
       'date', 'date_order', 'position', 'km_to_go', 'standing_remaining',
       'prone_remaining', 'probability_standing', 'probability_prone',
       'all_mistakes_prone', 'all_mistakes_standing', 'back_from_median',
       'gradient_back', 'gradient_standing', 'gradient_prone', 'gradient_pos'],
      dtype='object')

In [52]:
sixth_df_input = pd.DataFrame(columns = fifth_df_input.columns)
gamma_behind = 0.95

gamma_standing = 0.95

gamma_prone = 0.95

for sex in fifth_df_input.sex.unique():
    df_sex = fifth_df_input[fifth_df_input['sex'] == sex]
    for date in df_sex.date.unique():
        df_date = df_sex[df_sex['date'] == date]

        for distance in df_date.distance.unique():
            df_distance = df_date[df_date['distance'] == distance]
            
            behinds = np.array(df_distance['Behind'])

            probabilities_standing = np.array(df_distance['probability_standing'])

            probabilities_prone = np.array(df_distance['probability_prone'])

            probabilites_standing = np.array(df_distance['probability_standing'])

            behinds_list_distance_above = []
            behinds_list_distance_below = []

            standing_list_distance_above = []
            standing_list_distance_below = []

            prone_list_distance_above = []
            prone_list_distance_below = []

            for i in range(len(df_distance)):
                relative_behind = behinds - behinds[i]
                
                multplicative_behind_front = np.array([gamma_behind**(i + 1) for i in range(len(relative_behind[i:]))])
                multplicative_behind_back = np.array([gamma_behind**(i + 1) for i in range(len(relative_behind[:i]))])
                
                

                multiplicative_standing_front = np.array([gamma_standing**(i + 1) for i in range(len(relative_behind[i:]))])
                multiplicative_standing_back = np.array([gamma_standing**(i + 1) for i in range(len(relative_behind[:i]))])

                multiplicative_prone_front = np.array([gamma_prone**(i + 1) for i in range(len(relative_behind[i:]))])
                multiplicative_prone_back = np.array([gamma_prone**(i + 1) for i in range(len(relative_behind[:i]))])


                weighted_behind_above = np.sum(relative_behind[i:] * multplicative_behind_front)
                weighted_behind_below = np.sum(relative_behind[:i] * multplicative_behind_back)
                
                weighted_standing_above = np.sum(probabilities_standing[i:] * multiplicative_standing_front)
                weighted_standing_below = np.sum(probabilities_standing[:i] * multiplicative_standing_back)

                weighted_prone_above = np.sum(probabilities_prone[i:] * multiplicative_prone_front)
                weighted_prone_below = np.sum(probabilities_prone[:i] * multiplicative_prone_back)

                
                
                behinds_list_distance_above.append(weighted_behind_above)
                behinds_list_distance_below.append(weighted_behind_below)

                standing_list_distance_above.append(weighted_standing_above)
                standing_list_distance_below.append(weighted_standing_below)

                prone_list_distance_above.append(weighted_prone_above)
                prone_list_distance_below.append(weighted_prone_below)
                
                    

            
            df_distance['behind_above'] = behinds_list_distance_above
            df_distance['behind_below'] = behinds_list_distance_below
            
            df_distance['standing_above'] = standing_list_distance_above
            df_distance['standing_below'] = standing_list_distance_below

            df_distance['prone_above'] = prone_list_distance_above
            df_distance['prone_below'] = prone_list_distance_below

            sixth_df_input = pd.concat([sixth_df_input, df_distance])


In [ ]:
pd.read_csv()

In [61]:
sixth_df_input.columns


Index(['full_name', 'distance', 'Rank', 'sex', 'Behind', 'format', 'season',
       'date', 'date_order', 'position', 'km_to_go', 'standing_remaining',
       'prone_remaining', 'probability_standing', 'probability_prone',
       'all_mistakes_prone', 'all_mistakes_standing', 'back_from_median',
       'gradient_back', 'gradient_standing', 'gradient_prone', 'gradient_pos',
       'behind_above', 'behind_below', 'standing_above', 'standing_below',
       'prone_above', 'prone_below'],
      dtype='object')

In [91]:
sixth_df_input['date_order'] = sixth_df_input['date_order'].fillna(0)
sixth_df_input.dropna(inplace = True)

In [99]:
#sixth_df_input.to_csv('sixth_df_input.csv', index = False)

In [3]:
fourth_df_input = pd.read_csv('fourth_df_input.csv')
fourth_df_input

,full_name,distance,Rank,sex,Behind,format,season,date,date_order,position,km_to_go,standing_remaining,prone_remaining,probability_standing,probability_prone,all_mistakes_prone,all_mistakes_standing,back_from_median,gradient_back
0,Aita_Gasparin,1.1,53.0,women,21.2,individual,2016-2017,2016-11-30,1.0,53.0,13.9,2,2,0.616470,0.871039,0.0,0.0,0.82,0.00
1,Aita_Gasparin,1.5,53.0,women,34.2,individual,2016-2017,2016-11-30,1.0,63.0,13.5,2,2,0.616470,0.871039,0.0,0.0,0.82,0.00
2,Aita_Gasparin,2.3,53.0,women,40.1,individual,2016-2017,2016-11-30,1.0,62.0,12.7,2,2,0.616470,0.871039,0.0,0.0,0.82,0.00
3,Aita_Gasparin,3.0,53.0,women,173.0,individual,2016-2017,2016-11-30,1.0,71.0,12.0,2,1,0.000000,0.600000,2.0,0.0,0.82,0.00
4,Aita_Gasparin,4.1,53.0,women,187.9,individual,2016-2017,2016-11-30,1.0,71.0,10.9,2,1,0.000000,0.600000,2.0,0.0,0.82,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325509,Otto_Invenius,8.4,48.0,men,297.1,pursuit,2021-2022,2022-03-19,203.0,47.0,4.1,1,0,0.700000,0.933333,1.0,1.0,1.97,1.16
325510,Otto_Invenius,9.2,48.0,men,302.5,pursuit,2021-2022,2022-03-19,203.0,47.0,3.3,1,0,0.700000,0.933333,1.0,1.0,1.97,1.16
325511,Otto_Invenius,10.0,48.0,men,338.2,pursuit,2021-2022,2022-03-19,203.0,49.0,2.5,0,0,0.666667,0.933333,1.0,3.0,1.97,1.16
325512,Otto_Invenius,10.9,48.0,men,348.3,pursuit,2021-2022,2022-03-19,203.0,48.0,1.6,0,0,0.666667,0.933333,1.0,3.0,1.97,1.16


In [83]:
#fourth_df_input.to_csv('fourth_df_input.csv', index = False)

In [81]:
fourth_df_input[['full_name', 'gradient_back', 'back_from_median', 'date', 'season']][0:50]

,full_name,gradient_back,back_from_median,date,season
0,Aita_Gasparin,0.000000,0.820000,2016-11-30,2016-2017
1,Aita_Gasparin,0.000000,0.820000,2016-11-30,2016-2017
2,Aita_Gasparin,0.000000,0.820000,2016-11-30,2016-2017
3,Aita_Gasparin,0.000000,0.820000,2016-11-30,2016-2017
4,Aita_Gasparin,0.000000,0.820000,2016-11-30,2016-2017
5,Aita_Gasparin,0.000000,0.820000,2016-11-30,2016-2017
6,Aita_Gasparin,0.000000,0.820000,2016-11-30,2016-2017
7,Aita_Gasparin,0.000000,0.820000,2016-11-30,2016-2017
8,Aita_Gasparin,0.000000,0.820000,2016-11-30,2016-2017
9,Aita_Gasparin,0.000000,0.820000,2016-11-30,2016-2017


In [119]:
tmp = third_df_input[third_df_input['season'] == '2016-2017' ][third_df_input[third_df_input['season'] == '2016-2017' ]['full_name'] == 'Aita_Gasparin']

In [124]:
tmp_2 = {str(date) : np.mean(tmp[tmp['date'] == date].back_from_median) for date in tmp['date'].unique()}

In [134]:
tmp_3 = np.array(list(tmp_2.values()))
tmp_4 = pd.DataFrame(tmp_3, columns = ['tmp'])
np.gradient(tmp_4['tmp'].rolling(center = False, window = 5).mean())

array([            nan,             nan,             nan,             nan,
                   nan,  2.33369697e-01,  1.96664502e-01,  1.55696429e-01,
        1.02405303e-01,  5.21850982e-02,  1.10549329e-02, -1.24003594e-02,
       -1.20175207e-02,  1.61190850e-04])

In [146]:
np.mean(np.array([1.829357 - 1.602976, 1.602976  - 1.306667, 1.306667 - 1.154545, 1.154545  - 0.820000]))

0.25233925

In [166]:
np.mean(np.array([2.128242 - 1.829357, 1.829357 - 1.602976, 1.602976  - 1.306667, 1.306667 - 1.154545, 1.154545  - 0.820000]))

0.26164840000000006

In [144]:
tmp_4['tmp']

0     0.820000
1     1.154545
2     1.306667
3     1.602976
4     1.829357
5     2.128242
6     2.180000
7     2.247857
8     2.218750
9     2.237636
10    2.241814
11    2.176978
12    2.126876
13    2.219556
Name: tmp, dtype: float64

In [143]:
np.diff(tmp_4['tmp'])

array([ 0.33454545,  0.15212121,  0.29630952,  0.22638095,  0.29888528,
        0.05175758,  0.06785714, -0.02910714,  0.01888636,  0.00417782,
       -0.06483662, -0.05010159,  0.09267997])

In [164]:
tmp_4['tmp']

0     0.820000
1     1.154545
2     1.306667
3     1.602976
4     1.829357
5     2.128242
6     2.180000
7     2.247857
8     2.218750
9     2.237636
10    2.241814
11    2.176978
12    2.126876
13    2.219556
Name: tmp, dtype: float64

In [179]:
#tmp_fun = lambda x: np.mean(np.array([x[i] - x[i-1] for i in range(1, len(x))])) if lex(x) > 0 
tmp_ = [0]
tmp_ += [tmp_4['tmp'][1] - tmp_4['tmp'][0], np.mean([tmp_4['tmp'][2] - tmp_4['tmp'][1], tmp_4['tmp'][1] - tmp_4['tmp'][0]]),\
    np.mean([tmp_4['tmp'][3] - tmp_4['tmp'][2], tmp_4['tmp'][2] - tmp_4['tmp'][1], tmp_4['tmp'][1] - tmp_4['tmp'][0]]),\
        np.mean([tmp_4['tmp'][4] - tmp_4['tmp'][3], tmp_4['tmp'][3] - tmp_4['tmp'][2], tmp_4['tmp'][2] - tmp_4['tmp'][1], tmp_4['tmp'][1] - tmp_4['tmp'][0]])]
for i in range(len(tmp_4) - 5):
    
    tmp_.append(np.mean(np.array([tmp_4['tmp'][i + j + 1] - tmp_4['tmp'][i + j] for j in range(4)])))

tmp_
   

[0,
 0.33454545454545437,
 0.24333333333333312,
 0.26099206349206355,
 0.25233928571428565,
 0.25233928571428565,
 0.2434242424242425,
 0.21833333333333343,
 0.1612202380952379,
 0.09734821428571444,
 0.027348484848484955,
 0.015453546453546463,
 -0.01771989374262084,
 -0.022968504709576343]

In [182]:
tmp_dict = {list(tmp_2.keys())[i] : tmp_[i] for i in range(len(tmp_2))}
tmp_dict

{'2016-11-30': 0,
 '2016-12-03': 0.33454545454545437,
 '2016-12-09': 0.24333333333333312,
 '2016-12-16': 0.26099206349206355,
 '2016-12-17': 0.25233928571428565,
 '2017-01-14': 0.25233928571428565,
 '2017-02-10': 0.2434242424242425,
 '2017-02-12': 0.21833333333333343,
 '2017-02-15': 0.1612202380952379,
 '2017-03-02': 0.09734821428571444,
 '2017-03-04': 0.027348484848484955,
 '2017-03-10': 0.015453546453546463,
 '2017-03-17': -0.01771989374262084,
 '2017-03-18': -0.022968504709576343}

In [183]:
tmp['colonna_prova'] = tmp['date'].map(tmp_dict)

In [185]:
tmp[0:20]

,full_name,distance,Rank,sex,Behind,format,season,date,date_order,position,km_to_go,standing_remaining,prone_remaining,probability_standing,probability_prone,all_mistakes_prone,all_mistakes_standing,back_from_median,colonna_prova
0,Aita_Gasparin,1.1,53,women,21.2,individual,2016-2017,2016-11-30,1.0,53.0,13.9,2,2,0.615638,0.870678,0.0,0.0,0.82,0.000000
1,Aita_Gasparin,1.5,53,women,34.2,individual,2016-2017,2016-11-30,1.0,63.0,13.5,2,2,0.615638,0.870678,0.0,0.0,0.82,0.000000
2,Aita_Gasparin,2.3,53,women,40.1,individual,2016-2017,2016-11-30,1.0,62.0,12.7,2,2,0.615638,0.870678,0.0,0.0,0.82,0.000000
3,Aita_Gasparin,3.0,53,women,173.0,individual,2016-2017,2016-11-30,1.0,71.0,12.0,2,1,0.000000,0.600000,2.0,0.0,0.82,0.000000
4,Aita_Gasparin,4.1,53,women,187.9,individual,2016-2017,2016-11-30,1.0,71.0,10.9,2,1,0.000000,0.600000,2.0,0.0,0.82,0.000000
5,Aita_Gasparin,4.5,53,women,195.8,individual,2016-2017,2016-11-30,1.0,70.0,10.5,2,1,0.000000,0.600000,2.0,0.0,0.82,0.000000
6,Aita_Gasparin,5.3,53,women,203.0,individual,2016-2017,2016-11-30,1.0,70.0,9.7,2,1,0.000000,0.600000,2.0,0.0,0.82,0.000000
7,Aita_Gasparin,6.0,53,women,250.0,individual,2016-2017,2016-11-30,1.0,44.0,9.0,1,1,0.800000,0.600000,2.0,1.0,0.82,0.000000
8,Aita_Gasparin,7.1,53,women,275.0,individual,2016-2017,2016-11-30,1.0,49.0,7.9,1,1,0.800000,0.600000,2.0,1.0,0.82,0.000000
9,Aita_Gasparin,7.5,53,women,283.8,individual,2016-2017,2016-11-30,1.0,51.0,7.5,1,1,0.800000,0.600000,2.0,1.0,0.82,0.000000


In [159]:
pd.Series(tmp_4['tmp']).rolling(center = False, window = 5).agg(tmp_fun)

KeyError: 0

In [98]:
len(df_input)

325523

In [79]:
df_input.iloc[:50 :]

,full_name,distance,Rank,sex,Behind,format,season,date,date_order,position,km_to_go,standing_remaining,prone_remaining,probability_standing,probability_prone,all_mistakes_prone,all_mistakes_standing,back_from_median
0,Aita_Gasparin,1.1,53,women,21.2,individual,2016-2017,2016-11-30,1.0,53.0,13.9,2,2,NaN,NaN,NaN,NaN,NaN
1,Aita_Gasparin,1.5,53,women,34.2,individual,2016-2017,2016-11-30,1.0,63.0,13.5,2,2,NaN,NaN,NaN,NaN,NaN
2,Aita_Gasparin,2.3,53,women,40.1,individual,2016-2017,2016-11-30,1.0,62.0,12.7,2,2,NaN,NaN,NaN,NaN,NaN
3,Aita_Gasparin,3.0,53,women,173.0,individual,2016-2017,2016-11-30,1.0,71.0,12.0,2,1,0.0,0.6,2.0,0.0,0.82
4,Aita_Gasparin,4.1,53,women,187.9,individual,2016-2017,2016-11-30,1.0,71.0,10.9,2,1,NaN,NaN,NaN,NaN,NaN
5,Aita_Gasparin,4.5,53,women,195.8,individual,2016-2017,2016-11-30,1.0,70.0,10.5,2,1,NaN,NaN,NaN,NaN,NaN
6,Aita_Gasparin,5.3,53,women,203.0,individual,2016-2017,2016-11-30,1.0,70.0,9.7,2,1,NaN,NaN,NaN,NaN,NaN
7,Aita_Gasparin,6.0,53,women,250.0,individual,2016-2017,2016-11-30,1.0,44.0,9.0,1,1,0.8,0.6,2.0,1.0,0.82
8,Aita_Gasparin,7.1,53,women,275.0,individual,2016-2017,2016-11-30,1.0,49.0,7.9,1,1,NaN,NaN,NaN,NaN,NaN
9,Aita_Gasparin,7.5,53,women,283.8,individual,2016-2017,2016-11-30,1.0,51.0,7.5,1,1,NaN,NaN,NaN,NaN,NaN


In [47]:
second_df_loop[['full_name', 'season','percentage_back', 'back_from_median']][350:400]

,full_name,season,percentage_back,back_from_median
350,Erik_Lesser,2017-2018,-1.30,-1.884375
351,Erik_Lesser,2017-2018,-1.30,-1.884375
352,Erik_Lesser,2017-2018,-3.18,-1.960588
353,Erik_Lesser,2017-2018,-3.18,-1.960588
354,Erik_Lesser,2017-2018,-0.24,-1.865000
355,Erik_Lesser,2017-2018,-0.24,-1.865000
356,Erik_Lesser,2017-2018,-0.24,-1.865000
357,Erik_Lesser,2017-2018,-0.24,-1.865000
358,Erik_Lesser,2017-2018,-2.05,-1.874737
359,Erik_Lesser,2017-2018,-2.05,-1.874737


In [43]:
pd.read_csv('/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/women/2018-2019_w/Hochfilzen/analysis_Hochfilzen_sprint.txt', sep ='\t')

,Rank,Bib,Family Name,Given Name,Nation,Total Time,Course\nTime\nRank,back from\naverage\n(in %),back from\ntop10 average\n(in %),back from\ntop30 average\n(in %),back from\nmedian\n(in %),back from\ntop10 median\n(in %),back from\ntop30 median\n(in %),km/h,Course\nTime\nzScore
0,1,56,Wierer,Dorothea,ITA,18:58.0,3,-5.67,-0.39,-1.85,-5.10,-0.57,-2.27,22.93,-1.47
1,2,22,Makarainen,Kaisa,FIN,18:34.0,1,-7.66,-2.49,-3.92,-7.10,-2.66,-4.33,23.43,-1.99
2,3,61,Yurlova-Percht,Ekaterina,RUS,19:29.7,22,-3.04,2.38,0.89,-2.45,2.20,0.45,22.31,-0.79
3,4,69,Vittozzi,Lisa,ITA,19:04.0,5,-5.17,0.13,-1.33,-4.60,-0.04,-1.76,22.81,-1.34
4,5,31,Hojnisz-Starega,Monika,POL,19:24.7,16,-3.46,1.94,0.46,-2.87,1.76,0.02,22.41,-0.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,94,101,Klemencic,Polona,SLO,21:08.8,87,5.17,11.05,9.44,5.81,10.86,8.96,20.57,1.34
94,95,43,Einfalt,Lea,SLO,20:36.2,75,2.47,8.20,6.62,3.09,8.01,6.16,21.11,0.64
95,96,48,Tang,Jialin,CHN,21:12.9,91,5.51,11.41,9.79,6.15,11.22,9.31,20.50,1.43
96,97,100,Rasimoviciute,Diana,LTU,23:07.8,98,15.03,21.47,19.70,15.74,21.26,19.18,18.81,3.90


In [65]:
#second_df_loop.to_csv('second_df_loop.csv', index = False)

In [28]:
second_df_loop = pd.read_csv('second_df_loop.csv')

array(['Simon_Schempp', 'Erik_Lesser', 'Martin_Fourcade',
       'Jean Guillaume_Beatrix', 'Ole Einar_Bjoerndalen', 'Benedikt_Doll',
       'Emil Hegle_Svendsen', 'Anton_Babikov', 'Michal_Krcmar',
       'Benjamin_Weger', 'Maxim_Tsvetkov', 'Dominik_Landertinger',
       'Ondrej_Moravec', 'Anton_Shipulin', 'Matvey_Eliseev',
       'Lukas_Hofer', 'Dmitry_Malyshko', 'Arnd_Peiffer', 'Michael_Roesch',
       "Henrik_L'Abee-Lund", 'Vladimir_Iliev', 'Julian_Eberhard',
       'Sergii_Semenov', 'Daniel_Mesotitsch', 'Volodymyr_Siemakov',
       'Simon_Desthieux', 'Dmytro_Pidruchnyi', 'Dominik_Windisch',
       'Michal_Slesingr', 'Artem_Pryma', 'Fabien_Claude',
       'Quentin_Fillon Maillet', 'Tim_Burke',
       'Vetle Sjaastad_Christiansen', 'Yury_Shopin', 'Antonin_Guigonnat',
       'Adam_Vaclavik', 'Russell_Currier', 'Matthias_Bischl',
       'Sergey_Bocharnikov', 'Dimitar_Gerdzhikov', 'Matej_Kazar',
       'Erlend_Bjoentegaard', 'Remus_Faur', 'Fredrik_Lindstroem',
       'Giuseppe_Montello',

In [67]:
second_df_loop

,Rank,Bib,Family Name,Given Name,Nation,Cumulative Time,Loop Time,Course Time\n,Course Time,Range Time,...,date_order,standing_remaining,prone_remaining,full_name,shootings_done,probability_standing,probability_prone,all_mistakes_prone,all_mistakes_standing,percentage_back
0,46,9,Schempp,Simon,GER,+18.4,+18.4,NaN,+10.1,10.9,...,2.0,2,1,Simon_Schempp,1,0.0,1.000000,0.0,0.0,-5.06
1,46,9,Schempp,Simon,GER,+4:48.6,+4:36.0,NaN,+9.4,32.6,...,2.0,1,1,Simon_Schempp,2,0.2,1.000000,0.0,4.0,-5.06
2,46,9,Schempp,Simon,GER,+5:44.3,+1:03.4,NaN,+3.2,9.4,...,2.0,1,0,Simon_Schempp,3,0.2,0.900000,1.0,4.0,-5.06
3,46,9,Schempp,Simon,GER,+5:37.6,+2:03.7,NaN,9:28.3,+9.7,...,2.0,0,0,Simon_Schempp,4,0.4,0.900000,1.0,6.0,-5.06
4,21,22,Schempp,Simon,GER,+36.7,+36.7,NaN,+7.7,11.2,...,3.0,1,0,Simon_Schempp,1,0.4,0.866667,1.0,0.0,-3.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63585,85,73,Horodna,Olena,UKR,+4:04.3,+2:36.0,NaN,+1:15.8,7.6,...,200.0,0,0,Olena_Horodna,2,0.4,0.800000,1.0,3.0,10.96
63586,99,110,Arnekleiv,Juni,NOR,+41.2,+41.2,NaN,+36.7,7.2,...,184.0,1,0,Juni_Arnekleiv,1,0.0,1.000000,0.0,0.0,7.56
63587,99,110,Arnekleiv,Juni,NOR,+2:11.8,+1:30.6,NaN,+1:02.0,10.1,...,184.0,0,0,Juni_Arnekleiv,2,0.8,1.000000,0.0,1.0,7.56
63588,106,106,Dimitrova,Valentina,BUL,+1:07.4,+1:07.4,NaN,+1:03.1,7.7,...,184.0,1,0,Valentina_Dimitrova,1,0.0,1.000000,0.0,0.0,11.66


In [33]:
 ['back from\nmedian\n(in %)']

['back from\nmedian\n(in %)']

In [41]:
print(len(all_mistakes_standings)/2, len(second_df_loop))

66588.5 63590


In [40]:
#second_df_loop.to_csv('second_df_loop.csv', index = False)

In [41]:
second_df_loop = pd.read_csv('second_df_loop.csv')

In [30]:
second_df_loop

,Rank,Bib,Family Name,Given Name,Nation,Cumulative Time,Loop Time,Course Time\n,Course Time,Range Time,...,position,Ski Time,km_to_go,date_order,standing_remaining,prone_remaining,full_name,shootings_done,probability_standing,probability_prone
0,46,9,Schempp,Simon,GER,+18.4,+18.4,NaN,+10.1,10.9,...,6.0,+18.4,16.000000,2.0,2,1,Simon_Schempp,1,0.0,1.000000
1,46,9,Schempp,Simon,GER,+4:48.6,+4:36.0,NaN,+9.4,32.6,...,75.0,+48.6,12.000000,2.0,1,1,Simon_Schempp,2,0.2,1.000000
2,46,9,Schempp,Simon,GER,+5:44.3,+1:03.4,NaN,+3.2,9.4,...,61.0,+44.3,8.000000,2.0,1,0,Simon_Schempp,3,0.2,0.900000
3,46,9,Schempp,Simon,GER,+5:37.6,+2:03.7,NaN,9:28.3,+9.7,...,51.0,+37.6,4.000000,2.0,0,0,Simon_Schempp,4,0.4,0.900000
4,21,22,Schempp,Simon,GER,+36.7,+36.7,NaN,+7.7,11.2,...,45.0,NaN,6.666667,3.0,1,0,Simon_Schempp,1,0.4,0.866667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63585,85,73,Horodna,Olena,UKR,+4:04.3,+2:36.0,NaN,+1:15.8,7.6,...,85.0,NaN,2.500000,200.0,0,0,Olena_Horodna,2,0.4,0.800000
63586,99,110,Arnekleiv,Juni,NOR,+41.2,+41.2,NaN,+36.7,7.2,...,55.0,NaN,5.000000,184.0,1,0,Juni_Arnekleiv,1,0.0,1.000000
63587,99,110,Arnekleiv,Juni,NOR,+2:11.8,+1:30.6,NaN,+1:02.0,10.1,...,81.0,NaN,2.500000,184.0,0,0,Juni_Arnekleiv,2,0.8,1.000000
63588,106,106,Dimitrova,Valentina,BUL,+1:07.4,+1:07.4,NaN,+1:03.1,7.7,...,87.0,NaN,5.000000,184.0,1,0,Valentina_Dimitrova,1,0.0,1.000000


In [45]:
tmp = second_df_loop[second_df_loop['full_name'] == 'Martin_Fourcade']
tmp[tmp['season'] == '2016-2017']

,Rank,Bib,Family Name,Given Name,Nation,Cumulative Time,Loop Time,Course Time\n,Course Time,Range Time,...,position,Ski Time,km_to_go,date_order,standing_remaining,prone_remaining,full_name,shootings_done,probability_standing,probability_prone
1540,1,10,Fourcade,Martin,FRA,9:50.3,9:50.3,NaN,9:03.3,2.6,...,1.0,9:50.3,16.000000,2.0,2,1,Martin_Fourcade,1,0.000000,1.000000
1643,1,10,Fourcade,Martin,FRA,+2:18.1,+2:23.9,NaN,9:04.8,29.9,...,15.0,+18.1,12.000000,2.0,1,1,Martin_Fourcade,2,0.600000,1.000000
1746,1,10,Fourcade,Martin,FRA,+2:10.4,10:09.6,NaN,+4.9,4.3,...,4.0,+10.4,8.000000,2.0,1,0,Martin_Fourcade,3,0.600000,1.000000
1849,1,10,Fourcade,Martin,FRA,42:37.6,10:19.1,NaN,+5.1,+1.0,...,1.0,40:37.6,4.000000,2.0,0,0,Martin_Fourcade,4,0.800000,1.000000
1330,1,11,Fourcade,Martin,FRA,8:19.5,8:19.5,NaN,7:23.7,4.7,...,1.0,NaN,6.666667,3.0,1,0,Martin_Fourcade,1,0.800000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4585,2,2,Fourcade,Martin,FRA,+21.4,+26.3,NaN,+13.1,3.5,...,3.0,NaN,2.500000,34.0,0,0,Martin_Fourcade,4,0.873171,0.931707
4296,1,1,Fourcade,Martin,FRA,+21.0,+21.0,NaN,+0.2,23.2,...,18.0,NaN,12.000000,35.0,2,1,Martin_Fourcade,1,0.873171,0.933333
4326,1,1,Fourcade,Martin,FRA,+3.6,7:17.7,NaN,6:22.4,5.0,...,6.0,NaN,9.000000,35.0,2,0,Martin_Fourcade,2,0.873171,0.934884
4356,1,1,Fourcade,Martin,FRA,22:28.4,+0.5,NaN,+17.7,2.3,...,1.0,NaN,6.000000,35.0,1,0,Martin_Fourcade,3,0.876190,0.934884


In [ ]:
pd.read_csv('/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/women/2021-2022_w/Oslo_Holmenkollen/shooting_Oslo_Holmenkollen_mass_start.txt', sep = '\t')

,Rank,Bib,Family Name,Given Name,Nation,Lane,Shooting 1,Time,Lane.1,Shooting 2,Time.1,Lane.2,Shooting 3,Time.2,Lane.3,Shooting 4,Time.3
0,1,11,Braisaz-Bouchet,Justine,FRA,11,15.5s 3.3s 2.8s 2.6s 4.3s \n50321,32.0,13,19.1s 3.2s 2.6s 2.7s 2.9s \n54021,33.6,12,11.7s 2.3s 2.3s 2.9s 3.3s \n54321,25.4,8,13.8s 2.1s 2.0s 2.4s 2.8s \n54321,25.4
1,2,18,Preuss,Franziska,GER,18,12.1s 1.9s 2.6s 2.2s 1.9s \n54321,24.0,2,17.5s 2.1s 2.4s 2.2s 1.9s \n54301,29.9,10,13.0s 2.8s 2.0s 1.8s 1.6s \n12345,23.9,3,14.2s 1.8s 2.0s 2.0s 2.0s \n12045,24.7
2,3,1,Roeiseland,Marte Olsbu,NOR,1,10.5s 2.3s 2.3s 2.0s 2.0s \n54321,22.4,1,14.3s 2.4s 2.6s 2.7s 3.2s \n54021,29.1,5,9.9s 2.5s 2.2s 1.8s 3.0s \n54321,22.6,2,12.1s 1.8s 1.8s 1.7s 3.1s \n04321,22.8
3,4,17,Persson,Linn,SWE,17,14.7s 2.5s 2.1s 1.9s 1.9s \n54321,25.7,5,13.0s 2.1s 2.1s 2.3s 2.1s \n54321,25.1,3,11.7s 2.0s 1.7s 2.9s 3.1s \n54021,24.7,7,11.8s 1.9s 2.9s 2.0s 2.5s \n04321,23.2
4,5,2,Oeberg,Elvira,SWE,2,15.6s 3.3s 2.3s 2.7s 2.4s \n54321,29.3,4,14.2s 3.1s 2.4s 2.2s 2.4s \n54321,27.7,1,12.2s 2.6s 2.2s 2.7s 2.4s \n50321,25.0,5,11.4s 2.9s 1.9s 2.0s 2.4s \n54301,23.1
5,6,14,Tandrevold,Ingrid Landmark,NOR,14,11.9s 2.8s 2.6s 2.7s 3.0s \n12045,26.7,18,14.1s 2.8s 2.6s 2.8s 5.1s \n12345,30.7,9,12.0s 2.6s 2.7s 2.3s 4.1s \n12345,25.5,4,12.7s 2.3s 2.2s 2.5s 2.3s \n12340,25.0
6,7,6,Herrmann,Denise,GER,6,14.9s 2.7s 2.7s 2.8s 2.9s \n02345,29.2,12,16.6s 2.6s 2.8s 3.1s 3.6s \n02345,31.8,11,13.9s 2.0s 2.1s 1.9s 2.0s \n54321,24.2,9,12.7s 1.6s 2.1s 1.9s 3.5s \n54301,23.8
7,8,3,Hauser,Lisa Theresa,AUT,3,12.0s 2.6s 2.5s 2.7s 2.5s \n12345,25.4,3,13.2s 2.4s 2.3s 2.3s 2.2s \n12345,26.0,2,10.8s 2.7s 2.3s 2.1s 2.2s \n12345,23.3,1,12.2s 2.4s 2.3s 1.8s 2.3s \n12340,24.0
8,9,26,Hettich,Janina,GER,26,10.1s 3.5s 2.4s 2.9s 2.6s \n54321,25.8,11,14.6s 4.2s 2.5s 2.3s 2.3s \n54321,29.1,7,12.8s 2.5s 2.3s 2.2s 2.6s \n54321,24.5,11,10.9s 2.5s 2.3s 2.0s 2.2s \n54321,23.2
9,10,13,Voigt,Vanessa,GER,13,12.2s 2.7s 2.7s 3.9s 3.6s \n12345,27.8,6,17.9s 2.8s 4.0s 4.6s 3.7s \n10345,36.4,15,16.7s 2.5s 3.0s 2.2s 2.4s \n54321,29.1,13,18.2s 5.1s 2.8s 2.6s 2.8s \n54321,33.5


In [ ]:
pd.read_csv('/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2021-2022_m/Oslo_Holmenkollen/shooting_Oslo_Holmenkollen_pursuit.txt',\
    sep = '\t')

,Rank,Bib,Family Name,Given Name,Nation,Lane,Shooting 1,Time,Lane.1,Shooting 2,Time.1,Lane.2,Shooting 3,Time.2,Lane.3,Shooting 4,Time.3
0,1,5,Lesser,Erik,GER,4,11.6s 2.0s 1.8s 1.9s 1.9s \n54321,22.0,4,15.1s 2.0s 1.9s 1.7s 2.1s \n54321,26.3,2,12.0s 1.8s 1.7s 1.9s 1.7s \n54321,21.2,2,11.0s 1.8s 1.7s 2.0s 2.1s \n54321,20.9
1,2,2,Fillon Maillet,Quentin,FRA,2,15.1s 2.2s 2.2s 2.2s 2.1s \n12345,28.0,2,14.9s 2.5s 2.0s 2.0s 2.1s \n02345,26.5,6,10.1s 1.9s 1.5s 1.4s 1.6s \n12345,18.6,3,11.9s 1.8s 1.6s 1.5s 1.5s \n12345,20.2
2,3,1,Laegreid,Sturla Holm,NOR,1,13.3s 2.1s 2.0s 1.8s 1.9s \n54321,25.2,1,14.3s 1.9s 1.9s 2.8s 2.1s \n54021,25.7,1,11.6s 2.0s 2.0s 1.9s 1.9s \n54321,30.2,1,12.9s 2.0s 2.0s 2.3s 2.7s \n50021,24.2
3,4,3,Samuelsson,Sebastian,SWE,3,13.4s 2.6s 2.3s 2.1s 2.3s \n12345,26.2,3,14.3s 2.5s 2.4s 2.3s 2.4s \n12345,26.6,3,10.2s 2.5s 2.5s 2.3s 2.1s \n04301,22.1,5,10.6s 3.0s 2.7s 2.0s 3.7s \n54321,24.6
4,5,7,Christiansen,Vetle Sjaastad,NOR,7,14.3s 2.7s 2.4s 2.3s 2.6s \n54321,27.0,6,14.8s 2.6s 2.4s 2.6s 2.6s \n54321,28.1,5,12.9s 2.4s 1.9s 2.1s 2.0s \n54320,23.9,4,12.8s 2.5s 2.0s 1.9s 2.0s \n04321,23.9
5,6,10,Ponsiluoma,Martin,SWE,10,14.5s 3.6s 3.9s 2.8s 2.6s \n54300,31.0,15,15.3s 3.4s 2.1s 2.1s 2.3s \n54320,27.8,19,10.9s 2.1s 1.9s 1.8s 2.0s \n54321,21.3,9,11.9s 2.1s 1.9s 1.8s 1.9s \n54321,21.5
6,7,16,Bakken,Sivert Guttorm,NOR,14,13.9s 2.4s 2.8s 2.4s 2.7s \n54321,26.6,12,14.0s 2.1s 2.4s 2.2s 4.8s \n54321,28.3,10,13.0s 2.7s 1.9s 2.2s 1.8s \n54021,23.8,11,11.6s 3.3s 2.6s 2.5s 2.5s \n54321,24.4
7,8,9,Andersen,Aleksander Fjeld,NOR,8,16.2s 2.4s 2.3s 2.8s 3.0s \n54321,29.6,8,16.1s 2.0s 2.7s 2.2s 3.0s \n54321,29.1,7,14.6s 1.7s 1.5s 1.9s 2.1s \n50321,24.1,8,13.4s 1.8s 3.4s 1.8s 1.8s \n54301,24.4
8,9,4,Doll,Benedikt,GER,6,12.5s 2.9s 3.4s 2.2s 3.1s \n04321,27.6,9,12.6s 2.8s 2.3s 2.3s 2.4s \n54321,26.2,9,12.7s 2.6s 2.6s 2.1s 4.2s \n54321,26.8,6,11.5s 2.1s 3.0s 2.2s 2.2s \n50301,24.1
9,10,15,Strolia,Vytautas,LTU,16,15.8s 2.6s 2.6s 2.5s 2.4s \n54321,28.8,13,17.8s 2.9s 2.7s 2.2s 2.7s \n54320,31.1,18,10.2s 2.1s 1.7s 2.0s 2.2s \n54321,20.5,14,11.2s 2.3s 1.7s 1.7s 2.0s \n54321,21.0


In [ ]:
tmp = df_loops[df_loops['full_name'] == 'Martin_Fourcade']
tmp_2 = tmp[tmp['season'] == '2016-2017']
percentaces = [0]
hits = [0]
tryes = [0]

hits = {'standing': [0], 'prone': [0]}
tryes = {'standing': [0], 'prone': [0]}



for i in range(len(tmp_2)):
    
    #print(f'/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/{tmp_2.iloc[i, :].sex}/{tmp_2.iloc[i, :].season}_{tmp_2.iloc[i, :].sex}/{tmp_2.iloc[i, :].venue}/shooting_{tmp_2.iloc[i, :].venue}_{tmp_2.iloc[i, :].format}.txt')
    df = pd.read_csv(f'/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/{tmp_2.iloc[i, :].sex}/{tmp_2.iloc[i, :].season}_{tmp_2.iloc[i, :].sex[0]}/{tmp_2.iloc[i, :].venue}/shooting_{tmp_2.iloc[i, :].venue}_{tmp_2.iloc[i, :].format}.txt', sep = '\t')
    
    

    df['full_name'] = df['Given Name'] + '_' + df['Family\xa0Name']
    df['full_name'][df['full_name'] == 'Alexandr_Loginov'] = 'Alexander_Loginov'

    entry = df[df['full_name'] == tmp_2.iloc[i, :]['full_name']]
    shooting_string = entry[f'Shooting {tmp_2.iloc[i, :].shootings_done}']
    #print(str(shooting_string).split('\\n')[1])
    #print(str(shooting_string).split('\\n')[1], '#############')
    shooting_string = str(shooting_string).split('\\n')[1][:5]
    
    hits_ = 5 - shooting_string.count('0')
    #print(format_to_ranges[str(tmp_2.iloc[i, :].format)][tmp_2.iloc[i, :].shootings_done])
    
    hits[format_to_ranges[str(tmp_2.iloc[i, :].format)][tmp_2.iloc[i, :].shootings_done - 1]].\
        append(hits[format_to_ranges[str(tmp_2.iloc[i, :].format)][tmp_2.iloc[i, :].shootings_done - 1]][-1] + hits_)
    
    tryes[format_to_ranges[str(tmp_2.iloc[i, :].format)][tmp_2.iloc[i, :].shootings_done - 1]].\
        append(tryes[format_to_ranges[str(tmp_2.iloc[i, :].format)][tmp_2.iloc[i, :].shootings_done - 1]][-1] + 5)


/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_1993/4275802510.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_name'][df['full_name'] == 'Alexandr_Loginov'] = 'Alexander_Loginov'
/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_1993/4275802510.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_name'][df['full_name'] == 'Alexandr_Loginov'] = 'Alexander_Loginov'
/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_1993/4275802510.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http

In [ ]:
np.array(hits['standing'])/np.array(tryes['standing'])

/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_1993/3943201388.py:1: RuntimeWarning: invalid value encountered in true_divide
  np.array(hits['standing'])/np.array(tryes['standing'])


array([       nan, 0.6       , 0.8       , 0.86666667, 0.9       ,
       0.8       , 0.83333333, 0.85714286, 0.875     , 0.88888889,
       0.88      , 0.89090909, 0.86666667, 0.87692308, 0.85714286,
       0.86666667, 0.8625    , 0.84705882, 0.84444444, 0.84210526,
       0.83      , 0.82857143, 0.83636364, 0.84347826, 0.83333333,
       0.84      , 0.83846154, 0.83703704, 0.84285714, 0.84827586,
       0.84666667, 0.8516129 , 0.85625   , 0.86060606, 0.86470588,
       0.86857143, 0.86666667, 0.86486486, 0.86842105, 0.87179487,
       0.875     , 0.87804878, 0.88095238, 0.87906977])

In [ ]:
np.array(hits['prone'])/np.array(tryes['prone'])

/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_1993/1724587791.py:1: RuntimeWarning: invalid value encountered in true_divide
  np.array(hits['prone'])/np.array(tryes['prone'])


array([       nan, 1.        , 1.        , 1.        , 0.95      ,
       0.96      , 0.96666667, 0.97142857, 0.975     , 0.95555556,
       0.96      , 0.92727273, 0.93333333, 0.93846154, 0.94285714,
       0.94666667, 0.95      , 0.94117647, 0.94444444, 0.94736842,
       0.95      , 0.95238095, 0.94545455, 0.93913043, 0.94166667,
       0.944     , 0.94615385, 0.94074074, 0.93571429, 0.93103448,
       0.93333333, 0.92258065, 0.925     , 0.92727273, 0.92941176,
       0.92571429, 0.92777778, 0.92972973, 0.93157895, 0.93333333,
       0.935     , 0.93658537, 0.93809524, 0.93488372])

In [ ]:
format_to_ranges ={'individual': ['prone', 'standing', 'prone', 'standing'], 'sprint': ['prone', 'standing'], 'pursuit': ['prone', 'prone', 'standing',\
     'standing'],\
    'mass_start' : ['prone', 'prone', 'standing', 'standing'], 'sprint_2': ['prone', 'standing'],\
         'pursuit_2': ['prone', 'prone', 'standing', 'standing']}

In [ ]:
df_loops[df_loops['full_name'] == 'Alexander_Loginov']

,Rank,Bib,Family Name,Given Name,Nation,Cumulative Time,Loop Time,Course Time\n,Course Time,Range Time,...,Behind,season,venue,position,Ski Time,km_to_go,date_order,standing_remaining,prone_remaining,full_name
3533,72,31,Loginov,Alexander,RUS,+3:28.7,+3:28.7,NaN,+24.8,17.9,...,208.7,2016-2017,Hochfilzen,96.0,+35.3,15.000,26.0,1,2,Alexander_Loginov
3633,72,31,Loginov,Alexander,RUS,+5:35.8,+2:20.5,NaN,+13.5,8.3,...,335.8,2016-2017,Hochfilzen,91.0,+55.8,10.000,26.0,1,1,Alexander_Loginov
3733,72,31,Loginov,Alexander,RUS,+6:13.3,+1:14.7,NaN,+19.5,3.2,...,373.3,2016-2017,Hochfilzen,85.0,+1:10.7,5.000,26.0,0,1,Alexander_Loginov
3833,72,31,Loginov,Alexander,RUS,+7:14.3,+1:23.8,NaN,+21.9,3.0,...,434.3,2016-2017,Hochfilzen,73.0,+1:34.5,0.000,26.0,0,0,Alexander_Loginov
4451,36,38,Loginov,Alexander,RUS,+1:48.1,+31.1,NaN,+11.9,5.7,...,108.1,2016-2017,Oslo_Holmenkollen,35.0,NaN,9.375,34.0,1,2,Alexander_Loginov
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57163,2,32,Loginov,Alexandr,RUS,+48.1,+2.8,NaN,+5.0,4.0,...,48.1,2021-2022,Ruhpolding,4.0,NaN,6.250,186.0,0,2,Alexander_Loginov
57221,2,32,Loginov,Alexandr,RUS,+59.0,+24.6,NaN,+3.1,4.7,...,59.0,2021-2022,Ruhpolding,8.0,NaN,3.125,186.0,0,1,Alexander_Loginov
57279,2,32,Loginov,Alexandr,RUS,+34.0,+2.5,NaN,+4.2,6.2,...,34.0,2021-2022,Ruhpolding,3.0,NaN,0.000,186.0,0,0,Alexander_Loginov
57367,32,6,Loginov,Alexandr,RUS,+5.9,+5.9,NaN,+9.4,8.0,...,5.9,2021-2022,Ruhpolding,7.0,NaN,5.000,185.0,0,1,Alexander_Loginov


In [ ]:
for i in range(len(df_loops[df_loops['full_name'] == 'Alexander_Loginov'])):
    
    print(len(format_to_ranges[df_loops[df_loops['full_name'] == 'Alexander_Loginov'].iloc[i, :].format])\
         - df_loops[df_loops['full_name'] == 'Alexander_Loginov'].iloc[i, :].standing_remaining\
             - df_loops[df_loops['full_name'] == 'Alexander_Loginov'].iloc[i, :].prone_remaining)

1
2
3
4
1
2
3
4
1
2
1
2
3
4
1
2
1
2
3
4
1
2
3
4
1
2
1
2
3
4
1
2
1
2
3
4
1
2
3
4
1
2
3
4
1
2
1
2
3
4
1
2
3
4
1
2
1
2
3
4
1
2
1
2
3
4
1
2
3
4
1
2
3
4
1
2
1
2
3
4
1
2
1
2
3
4
1
2
3
4
1
2
1
2
3
4
1
2
3
4
1
2
1
2
3
4
1
2
3
4
1
2
3
4
1
2
1
2
3
4
1
2
1
2
3
4
1
2
1
2
3
4
1
2
3
4
1
2
1
2
3
4
1
2
1
2
3
4
1
2
3
4
1
2
1
2
1
2
3
4
1
2
3
4
1
2
1
2
1
2
3
4
1
2
3
4
1
2
1
2
3
4
1
2
1
2
3
4
1
2
3
4
1
2
1
2
3
4
1
2
1
2
3
4
1
2
3
4
1
2
1


KeyboardInterrupt: 

In [ ]:
tmp = df_predictor[df_predictor['full_name'] == 'Simon_Schempp']
print(tmp.iloc[0,:].sex, tmp.iloc[0,:].season, tmp.iloc[0,:].venue)

men 2016-2017 Oberhof


In [ ]:
tmp = tmp[tmp['season'] == '2017-2018']
tmp.sort_values(by = 'date_order', ascending=True)
tmp

,Rank,Bib,Family Name,Given Name,Nation,Sector Time,Sector Rank,Net Time,Behind,sex,...,position,venue,vRank,Unnamed: 0,Day Time,km_to_go,date_order,standing_remaining,prone_remaining,full_name
42336,34,7,Schempp,Simon,GER,33.3,8,33.3,1.2,men,...,8.0,Oberhof,NaN,NaN,NaN,9.8,48,1,1,Simon_Schempp
42439,34,7,Schempp,Simon,GER,1:19.9,9,1:53.2,4.8,men,...,9.0,Oberhof,NaN,NaN,NaN,9.2,48,1,1,Simon_Schempp
42542,34,7,Schempp,Simon,GER,1:32.8,10,3:26.0,6.8,men,...,10.0,Oberhof,NaN,NaN,NaN,8.3,48,1,1,Simon_Schempp
42645,34,7,Schempp,Simon,GER,53.8,12,4:19.8,10.2,men,...,12.0,Oberhof,NaN,NaN,NaN,8.0,48,1,1,Simon_Schempp
42748,34,7,Schempp,Simon,GER,1:22.3,66,11:32.2,65.5,men,...,66.0,Oberhof,NaN,NaN,NaN,5.9,48,0,1,Simon_Schempp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64153,36,21,Schempp,Simon,GER,41.2,40,45:14.6,218.8,men,...,40.0,PyeongChang,NaN,NaN,NaN,3.4,60,0,0,Simon_Schempp
64239,36,21,Schempp,Simon,GER,2:45.1,39,47:59.7,220.8,men,...,39.0,PyeongChang,NaN,NaN,NaN,1.7,60,0,0,Simon_Schempp
64325,36,21,Schempp,Simon,GER,42.3,39,48:42.0,223.5,men,...,39.0,PyeongChang,NaN,NaN,NaN,1.4,60,0,0,Simon_Schempp
64411,36,21,Schempp,Simon,GER,1:08.4,37,49:50.4,224.1,men,...,37.0,PyeongChang,NaN,NaN,NaN,0.8,60,0,0,Simon_Schempp


In [ ]:
print(tmp.iloc[0, :].sex, tmp.iloc[0, :].season, tmp.iloc[0, :].venue, tmp.iloc[0, :].format, tmp.iloc[5, :].distance ,\
    tmp.iloc[0, :].standing_remaining, tmp.iloc[0, :].prone_remaining)

men 2017-2018 Oberhof sprint 5.0 1 1


In [ ]:
standing_try = [0]
standing_mistakes = [0]

In [ ]:
def percentage_season(df, season):
    percentagea
    df_season = df[df['season'] == season]
    df_season = df_season.sort_values(by = 'date_order', ascending=True)
    
    


In [ ]:
tmp = pd.read_csv('/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Oberhof/shooting_Oberhof_mass_start.txt', sep = '\t')
tmp_2 = tmp['Shooting 1'][0].split('\n')[1]
max([int(i) for i in tmp_2])

5

In [ ]:
tmp

,Rank,Bib,Family Name,Given Name,Nation,Lane,Shooting 1,Time,Lane.1,Shooting 2,Time.1,Lane.2,Shooting 3,Time.2,Lane.3,Shooting 4,Time.3
0,1,3,Schempp,Simon,GER,3,15.5s 3.3s 2.6s 2.4s 2.6s \n12345,30.0,3,15.8s 2.7s 2.6s 2.8s 2.5s \n12345,31.0,2,14.4s 2.1s 2.2s 2.4s 2.9s \n05321,26.0,5,12.6s 2.4s 2.2s 3.9s 2.5s \n54321,26.0
1,2,6,Lesser,Erik,GER,6,12.1s 2.1s 1.9s 1.9s 1.8s \n54321,23.0,5,19.4s 2.2s 1.8s 1.9s 2.0s \n54321,31.0,5,11.9s 2.6s 2.2s 2.4s 2.6s \n54321,25.0,1,12.4s 2.5s 2.7s 3.9s 3.7s \n54301,29.0
2,3,1,Fourcade,Martin,FRA,1,13.5s 3.2s 2.7s 2.8s 3.1s \n54321,28.0,1,15.7s 2.6s 3.4s 2.5s 2.5s \n54321,30.0,1,9.5s 2.1s 3.2s 2.5s 2.5s \n05320,22.0,3,11.2s 2.4s 2.8s 3.1s 2.6s \n54321,24.0
3,4,16,Beatrix,Jean Guillaume,FRA,16,13.5s 2.2s 2.0s 2.1s 2.0s \n12345,26.0,11,16.2s 2.6s 2.5s 2.6s 2.6s \n12345,31.0,6,13.5s 3.1s 2.8s 2.9s 1.9s \n12340,27.0,6,12.0s 2.5s 1.8s 1.7s 1.9s \n12345,22.0
4,5,8,Bjoerndalen,Ole Einar,NOR,8,13.2s 3.0s 2.4s 2.4s 2.7s \n54321,26.0,2,14.3s 3.8s 3.2s 2.7s 3.0s \n54321,30.0,3,11.5s 2.2s 2.0s 1.8s 1.9s \n12345,23.0,2,11.3s 3.1s 2.0s 2.5s 3.0s \n12045,25.0
5,6,17,Doll,Benedikt,GER,17,12.2s 2.8s 2.1s 2.4s 2.2s \n54321,25.0,4,15.2s 2.7s 3.4s 2.8s 2.9s \n54301,32.0,9,14.6s 3.7s 2.9s 3.0s 2.6s \n54021,30.0,9,14.1s 3.1s 2.6s 2.7s 3.0s \n54321,28.0
6,7,7,Svendsen,Emil Hegle,NOR,7,13.2s 2.4s 2.1s 2.2s 2.4s \n54321,25.0,12,14.3s 2.1s 2.1s 2.4s 2.4s \n54321,28.0,4,15.5s 2.5s 1.8s 1.8s 2.5s \n50321,28.0,4,16.7s 2.7s 2.8s 2.8s 2.9s \n54320,31.0
7,8,10,Babikov,Anton,RUS,10,11.5s 2.8s 2.1s 2.4s 2.1s \n54321,25.0,10,13.7s 2.9s 2.8s 2.7s 2.7s \n54301,29.0,11,10.8s 2.4s 1.8s 1.9s 4.0s \n54321,23.0,8,10.5s 2.7s 2.8s 2.4s 2.5s \n54301,22.0
8,9,15,Krcmar,Michal,CZE,15,11.3s 3.7s 2.0s 1.9s 6.3s \n54321,28.0,6,14.4s 3.0s 2.1s 2.9s 2.7s \n04301,29.0,17,14.0s 2.4s 2.6s 1.9s 2.1s \n54301,25.0,13,14.9s 2.6s 2.4s 2.2s 2.2s \n54321,27.0
9,10,23,Weger,Benjamin,SUI,23,12.1s 3.2s 3.2s 2.8s 2.7s \n12345,28.0,15,15.7s 4.4s 2.7s 3.0s 3.2s \n12345,33.0,14,11.4s 2.0s 2.7s 2.4s 2.9s \n54301,23.0,14,12.3s 2.0s 1.9s 2.0s 1.8s \n54321,23.0


In [ ]:
i = 0
for _ in tmp:
    i+=1
    try:
        __ = to_seconds(_)
    except:
        print(_, i)

Rank 1
Bib 2
Family Name 3
Given Name 4
Nation 5
Lane 6
Shooting 1 7
Time 8
Lane.1 9
Shooting 2 10
Time.1 11
Lane.2 12
Shooting 3 13
Time.2 14
Lane.3 15
Shooting 4 16
Time.3 17


In [ ]:
tmp = df_predictor['Behind']
tmp = np.array(tmp)

In [ ]:
df_predictor['date_order'] = df_predictor['date_order'].astype({'date_order': 'int32'})
#input_predictor[np.isnan(input_predictor['date_order'])]

In [ ]:
tmp = tmp.sort_values(by = ['sex', 'race', 'distance'])
tmp.reset_index(inplace = True, drop = True)
tmp.to_csv('joint_df.csv', index = False)

AttributeError: 'numpy.ndarray' object has no attribute 'sort_values'

In [ ]:
indexes = {'men':int(0), 'women':int(0)}
indexes['men'] += 1
indexes['men']

1

In [ ]:
pd.read_csv('/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/women/2018-2019_w/Hochfilzen/metadata_2_Hochfilzen_pursuit.txt', sep = '\t', index_col = 0)

,Value (in m)
Total Course Length,7590
Height Difference,31.5
Max Climb,28
Total Climb,257
Date,2018-12-15


In [ ]:
input_predictor.reset_index(inplace = True, drop = True)

In [ ]:
input_predictor

,Rank,Bib,Family Name,Given Name,Nation,Sector Time,Sector Rank,Net Time,Behind,sex,distance,race,date,Unnamed: 8,Unnamed: 0,vRank,Unnamed: 9,Day Time
0,1,3,Schempp,Simon,GER,2:10.1,4,2:10.1,1.2,men,0.8,1.0,2017-01-08,NaN,NaN,NaN,NaN,NaN
1,2,6,Lesser,Erik,GER,2:11.0,7,2:11.0,2.1,men,0.8,1.0,2017-01-08,NaN,NaN,NaN,NaN,NaN
2,3,1,Fourcade,Martin,FRA,2:08.9,1,2:08.9,0.0,men,0.8,1.0,2017-01-08,NaN,NaN,NaN,NaN,NaN
3,4,16,Beatrix,Jean Guillaume,FRA,2:15.5,23,2:15.5,6.6,men,0.8,1.0,2017-01-08,NaN,NaN,NaN,NaN,NaN
4,5,8,Bjoerndalen,Ole Einar,NOR,2:11.3,8,2:11.3,2.4,men,0.8,1.0,2017-01-08,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,71,78,Sabule,Annija,LAT,46.4,70,23:27.5,+3:52.8,women,6.9,148.0,2022-03-18,NaN,NaN,NaN,NaN,14:17:26
71,72,66,Colebourn,Jillian Wei-Lin,AUS,46.4,72,24:26.9,+4:52.2,women,6.9,148.0,2022-03-18,NaN,NaN,NaN,NaN,14:12:25
72,73,64,Bulina,Sanita,LAT,55.4,73,24:40.4,+5:05.7,women,6.9,148.0,2022-03-18,NaN,NaN,NaN,NaN,14:11:40
73,74,50,Tanaka,Yurie,JPN,44.8,74,24:46.6,+5:11.9,women,6.9,148.0,2022-03-18,NaN,NaN,NaN,NaN,14:04:45


In [ ]:
tt = pd.read_csv('joint_df.csv')
tt[tt['Family Name'] == 'Loginov']

/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_799/731202880.py:1: DtypeWarning: Columns (0,1,6,14,15,17,18,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  tt = pd.read_csv('joint_df.csv')


,Rank,Bib,Family Name,Given Name,Nation,Sector Time,Sector Rank,Net Time,Behind,sex,...,Unnamed: 9,Day Time,Cumulative Time,Loop Time,Course Time,Course Time.1,Range Time,Shooting Time,Penalty Time,Ski Time
16473,72,31.0,Loginov,Alexander,RUS,1:41.2,26.0,1:41.2,4.5,men,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16573,72,31.0,Loginov,Alexander,RUS,48.9,19.0,2:30.1,4.4,men,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673,72,31.0,Loginov,Alexander,RUS,3:08.8,32.0,5:38.9,14.7,men,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16773,72,31.0,Loginov,Alexander,RUS,1:09.9,31.0,6:48.8,+18.2,men,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16873,72,31.0,Loginov,Alexander,RUS,2:04.5,96.0,14:39.5,+3:34.9,men,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321608,2,32,Loginov,Alexandr,RUS,1:38.2,4.0,17:37.7,+40.9,women,...,NaN,15:02:37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321825,2,32,Loginov,Alexandr,RUS,2:18.6,8.0,22:31.2,+57.0,women,...,NaN,15:07:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321936,2,32,Loginov,Alexandr,RUS,1:38.1,8.0,24:09.3,+56.7,women,...,NaN,15:09:09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
322100,2,32,Loginov,Alexandr,RUS,2:15.0,2.0,28:37.8,+22.0,women,...,NaN,15:13:37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
tmp ='/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Anterselva/metadata_2_Anterselva_mass_start.txt'
pd.read_csv(tmp, sep = '\t', index_col = 0).T['Date'][0]

'2017-01-22'

In [ ]:
np.sort(np.array(tmp))

array(['/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Oberhof/intermediates/mass_start/1.0km.txt',
       '/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Oberhof/intermediates/mass_start/3.8km.txt'],
      dtype='<U113')

In [ ]:
Hochfilzen pursuit men 2016-2017